## Importação das tabelas do GitHub

Para a importação das tabelas ser feita será preciso uma autenticação via Personal Access Token do GitHub para depois a criação de um Secret no Google Colab. Os passos a serem realizados são os seguintes:
- No GitHub ir a ***Settings***, e clicar em ***Developer Settings*** que se encontra na sidebar da esquerda
- Na sidebar da esquerda, em baixo de ***Personal Access Tokens*** clique em ***Tokens classic***
- Clique em ***Generate new token (classic)***, escolha o tipo de acessos assim como o periodo de validade que quer para a sua token e no fim clique ***Generate Token***
- Copie a token que foi gerada e crie um Secret no Google Colab com o nome the ***passToken*** e o valor da token que foi criada
- Dê acesso ao notebook para utilizar o secret

In [ ]:
pip install RapidFuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import geopandas as gpd
from shapely.geometry import Polygon, Point
import ipywidgets as widgets
from rapidfuzz import process, fuzz
from IPython.display import display, HTML
from math import radians, sin, cos, sqrt, atan2
import numpy as np
import unicodedata


In [ ]:
import requests
from io import BytesIO
import pandas as pd
from getpass import getpass
from google.colab import userdata

GITHUB_TOKEN = userdata.get('passToken')

FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/Contadores%20CM%20Maia.xlsx"

headers = {"Authorization": f"token {GITHUB_TOKEN}"}
response = requests.get(FILE_URL, headers=headers)

if response.status_code == 200:
  file_data = response.json()
  file_download_url = file_data["download_url"]

  file_response = requests.get(file_download_url)
  if file_response.status_code == 200:
    contadores = pd.read_excel(BytesIO(file_response.content))
  else:
    print("Failed to download this file:", file_response.status_code)

else:
  print("Failed to fetch file info:", response.status_code, response.json())

FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/Contadores%20CM%20Maia 2025.xlsx"

headers = {"Authorization": f"token {GITHUB_TOKEN}"}
response = requests.get(FILE_URL, headers=headers)

if response.status_code == 200:
  file_data = response.json()
  file_download_url = file_data["download_url"]

  file_response = requests.get(file_download_url)
  if file_response.status_code == 200:
    contadores_2 = pd.read_excel(BytesIO(file_response.content))
  else:
    print("Failed to download this file:", file_response.status_code)

else:
  print("Failed to fetch file info:", response.status_code, response.json())


FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/Levantamento_Equipamentos Escolares_Maia.xlsx"

response = requests.get(FILE_URL, headers=headers)

if response.status_code == 200:
  file_data = response.json()
  file_download_url = file_data["download_url"]

  file_response = requests.get(file_download_url)
  if file_response.status_code == 200:
    equipamentos = pd.read_excel(BytesIO(file_response.content))
  else:
    print("Failed to download this file:", file_response.status_code)

else:
  print("Failed to fetch file info:", response.status_code, response.json())

FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/MAPA%20DE%20BENS%20IMOVEIS.xlsx"

response = requests.get(FILE_URL, headers=headers)

if response.status_code == 200:
  file_data = response.json()
  file_download_url = file_data["download_url"]

  file_response = requests.get(file_download_url)
  if file_response.status_code == 200:
    imoveis = pd.read_excel(BytesIO(file_response.content), skiprows=3)
  else:
    print("Failed to download this file:", file_response.status_code)

else:
  print("Failed to fetch file info:", response.status_code, response.json())

FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/Mapa%20Certificação%20Energética.xlsx"

response = requests.get(FILE_URL, headers=headers)

if response.status_code == 200:
  file_data = response.json()
  file_download_url = file_data["download_url"]

  file_response = requests.get(file_download_url)
  if file_response.status_code == 200:
    energetica = pd.read_excel(BytesIO(file_response.content), skiprows=2)
  else:
    print("Failed to download this file:", file_response.status_code)

else:
  print("Failed to fetch file info:", response.status_code, response.json())

FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/Tabela%20de%20Edifícios%20-%20ISOV3.xlsx"

response = requests.get(FILE_URL, headers=headers)

if response.status_code == 200:
  file_data = response.json()
  file_download_url = file_data["download_url"]

  file_response = requests.get(file_download_url)
  if file_response.status_code == 200:
    edificios = pd.read_excel(BytesIO(file_response.content), skiprows=1)
  else:
    print("Failed to download this file:", file_response.status_code)

else:
  print("Failed to fetch file info:", response.status_code, response.json())

FILE_URL = "https://api.github.com/repos/pedroccpimenta/PI2025-DataHarmonization/contents/data/escolas_from_OSM.xlsx"

response = requests.get(FILE_URL, headers=headers)

## Filtragem de tabelas

### Filtragem da tabela "Contadores CM Maia"

***filtered_contadores*** : filtração da tabela ***contadores*** onde são retiradas colunas desnecessárias

In [ ]:
filtered_contadores = contadores.drop(columns=[
    'dtinstal_dcont', 'desig_cscon','dtsitua_fcons', 'nome_fenti','numcontrib_fenti', 'desig_fpost', 'entidade_contacto', 'freguesia_contacto',
    'rua_contacto', 'numpolicia_contacto', 'sonumero_contacto', 'desig_ffreg_contacto', 'desig_ffreg_contacto', 'desig_sitio_contacto',
    'cod_fpost_contacto', 'desig_fpost_contacto', 'zona_med_controlo', 'subsistsan', 'zona', 'ejr_numero', 'ejr_cod_lig',
    'ejr_num_max_lig', 'ejr_data_validacao', 'ejr_obs', 'cc_cod_tarif_ag', 'cc_desig_tarif_ag', 'cc_desig_tarif_disp_ag',
    'cc_desig_tarif_disp_lixo', 'cc_desig_tarif_lixo', 'cc_desig_tarif_disp_san', 'cc_desig_tarif_san', 'cc_desig_tarif_trh_ag',
    'cc_desig_tarif_trh_san', 'cc_desig_tarif_tcq', 'num_instalacao', 'li', 'nip', 'cc_coordenada_x', 'cc_coordenada_y',
    'cc_area_coletavel', 'contrato', 'data_contrato', 'cc_localizacao_contador', 'marca_contador', 'calibre_contador', 'cc_tp_contador',
    'cc_tp_tecno', 'email', 'telefone', 'sensib_fcons', 'data_fabrico_contador', 'modelo_contador', 'certifado_afer_contador',
    'inicio_arrend', 'fim_arrend', 'numlic_fcons', 'dtlicini_fcons', 'dtlicfim_fcons', 'tp_entidade'
])

filtered_contadores.head()

,chave_fcons,num_fcons,cod_carea,desig_carea,chave_carea,num_fcont,freguesia_ffreg,desigrua_fmora,numpolicia_fmora,sonumero_fmora,desig_ffreg,desig_fsiti,cod_fpost,tipo_consumidor,dt_instal_orig_contador,desig_edificio,Lat,Lon
0,28775,670,323,AGUAS SANTAS - LAS-3,186,I23JC095914,130601,PCT Ferreira de Castro,NaN,NaN,Águas Santas,NaN,4425-082,Autarquias Locais - DST,11/03/2024 00:00:00,Urb Meditur-Rega,NaN,NaN
1,29563,3460,589,MAIA - USM,323,I13JA287987,130619,R Samuel Gramaxo,RC A,43.0,Cidade da Maia,Maia,4470-213,Autarquias Locais,07/03/1994 00:00:00,NaN,-8.620295,41.224639
2,29564,3455,589,MAIA - USM,323,I13JA287996,130619,R Samuel Gramaxo,RC C,43.0,Cidade da Maia,Maia,4470-213,Autarquias Locais,07/03/1994 00:00:00,NaN,NaN,NaN
3,31770,1100,570,FOLGOSA - S.R.,310,D19BA063983,130603,R Central da Folgosa,NaN,NaN,Folgosa,NaN,4425-321,Autarquias locais - Escolas,26/03/2019 00:00:00,Escola EB1/JI Folgosa,NaN,NaN
4,31772,24600,313,AVIOSO SANTA MARIA - ISP-2,176,I23LA009785,130618,R Ferronho,NaN,NaN,Castêlo da Maia,Avioso (São Pedro),4475-703,Autarquias locais - Escolas,09/03/2023 00:00:00,Escola EB1/JI Ferronho,NaN,NaN


### Filtragem da tabela Contadores da Maia 2025

***filtered_contadores_2*** : filtração da tabela ***contadores 2025*** onde são retiradas colunas desnecessárias

In [ ]:
filtered_contadores_2 = contadores_2.drop(columns=[
    'dtinstal_dcont', 'desig_cscon','dtsitua_fcons', 'nome_fenti','numcontrib_fenti', 'desig_fpost', 'entidade_contacto', 'freguesia_contacto',
    'rua_contacto', 'numpolicia_contacto', 'sonumero_contacto', 'desig_ffreg_contacto', 'desig_ffreg_contacto', 'desig_sitio_contacto',
    'cod_fpost_contacto', 'desig_fpost_contacto', 'zona_med_controlo', 'subsistsan', 'zona', 'ejr_numero', 'ejr_cod_lig',
    'ejr_num_max_lig', 'ejr_data_validacao', 'ejr_obs', 'cc_cod_tarif_ag', 'cc_desig_tarif_ag', 'cc_desig_tarif_disp_ag',
    'cc_desig_tarif_disp_lixo', 'cc_desig_tarif_lixo', 'cc_desig_tarif_disp_san', 'cc_desig_tarif_san', 'cc_desig_tarif_trh_ag',
    'cc_desig_tarif_trh_san', 'cc_desig_tarif_tcq', 'num_instalacao', 'li', 'nip', 'cc_coordenada_x', 'cc_coordenada_y',
    'cc_area_coletavel', 'contrato', 'data_contrato', 'cc_localizacao_contador', 'marca_contador', 'calibre_contador', 'cc_tp_contador',
    'cc_tp_tecno', 'email', 'telefone', 'sensib_fcons', 'data_fabrico_contador', 'modelo_contador', 'certifado_afer_contador',
    'inicio_arrend', 'fim_arrend', 'numlic_fcons', 'dtlicini_fcons', 'dtlicfim_fcons', 'tp_entidade'
])

filtered_contadores_2.head()

,chave_fcons,num_fcons,cod_carea,desig_carea,chave_carea,num_fcont,freguesia_ffreg,desigrua_fmora,numpolicia_fmora,sonumero_fmora,...,cod_fpost,tipo_consumidor,dt_instal_orig_contador,desig_edificio,Lat,Lon,Mostra em base2b,Presente em waterlog,CPE,Nome da Escola
0,28775,670,323,AGUAS SANTAS - LAS-3,186,I23JC095914,130601,PCT Ferreira de Castro,NaN,NaN,...,4425-082,Autarquias Locais - DST,11/03/2024 00:00:00,Urb Meditur-Rega,-8.812110,41.218740,False,False,NaN,NaN
1,29563,3460,589,MAIA - USM,323,I13JA287987,130619,R Samuel Gramaxo,RC A,43.0,...,4470-213,Autarquias Locais,07/03/1994 00:00:00,NaN,-8.620295,41.224639,True,True,PT0002000065523872AA,NaN
2,29564,3455,589,MAIA - USM,323,I13JA287996,130619,R Samuel Gramaxo,RC C,43.0,...,4470-213,Autarquias Locais,07/03/1994 00:00:00,NaN,-8.620295,41.224639,True,True,PT0002000053093191SV,NaN
3,31770,1100,570,FOLGOSA - S.R.,310,D19BA063983,130603,R Central da Folgosa,NaN,NaN,...,4425-321,Autarquias locais - Escolas,26/03/2019 00:00:00,Escola EB1/JI Folgosa,-8.548260,41.268006,True,True,NaN,EB de Folgosa
4,31772,24600,313,AVIOSO SANTA MARIA - ISP-2,176,I23LA009785,130618,R Ferronho,NaN,NaN,...,4475-703,Autarquias locais - Escolas,09/03/2023 00:00:00,Escola EB1/JI Ferronho,-8.610829,41.275952,True,True,NaN,EB de Ferronho


### Filtragem da tabela "Levantamento_Equipamentos Escolares_Maia"

***filtered_equipamentos*** : filtração da tabela ***equipamentos*** onde entradas duplicadas são retiradas

In [ ]:
filtered_equipamentos = equipamentos.drop_duplicates(subset=["CodigoMISIescola", "NomeGrauEnsino"])
filtered_equipamentos = filtered_equipamentos.reset_index(drop=True)

filtered_equipamentos.head()

,NomeAgrupamento,CodigoMISIagrupamento,NomeEscola,CodigoMISIescola,NomeGrauEnsino
0,AE da Maia,152020,EB n.º 1 de Gueifães,295565,1º ciclo do ensino básico
1,AE da Maia,152020,EB n.º 1 de Gueifães,295565,educação pré-escolar
2,AE da Maia,152020,EB n.º 2 de Gueifães,287489,1º ciclo do ensino básico
3,AE da Maia,152020,EB n.º 2 de Gueifães,287489,educação pré-escolar
4,AE de Águas Santas,152961,EB de Corim,247418,1º ciclo do ensino básico


### Filtragem da tabela "MAPA DE BENS IMOVEIS"

***filtered_imoveis*** : filtração da tabela ***imoveis*** onde a coluna ***CONTA*** é removida devido a ter conteúdo similar à coluna ***COMPLEMENTAR***

***filtered_imoveis_2*** : filtração da tabela ***filtered_imoveis*** onde entradas que não contenham pelo menos uma das strings da lista ***keywords*** são retiradas

In [ ]:
filtered_imoveis = imoveis.drop(columns=['CONTA'])

keywords = [
    r"\bescola\b",
    r"\be\.?b\.?\s*\d?\b",  # Captura EB e as suas variações (pontos/espaços após as letras, números no final)
    r"\bcreche\b",
    r"\binfant[áa]rio\b",  # Captura Infantário com ou sem acento
    r"\bjardim\s*(?:de\s*)?inf[âa]ncia\b"  # Captura Jardim de infância com ou sem acento; "de" é opcional
]

include_pattern = "|".join(keywords)

filtered_imoveis_2 = filtered_imoveis[
    filtered_imoveis["DESIGNAÇAO"].astype(str).str.contains(include_pattern, case=False, na=False, regex=True)
]

filtered_imoveis.head()
filtered_imoveis_2

,N.º INVENTARIO,DESIGNAÇAO,COMPLEMENTAR,DATA AQUISIÇÃO/REGISTO,LOCALIZAÇÃO
46,34325.0,JARDIM NA ESQUINA DA RUA DA ESCOLA E RUA DE SE...,430291.0,2001-12-31,RUA DA ESCOLA E RUA DA SEARA
128,34408.0,"CANTEIROS JUNTO A ESCOLA, EM NOGUEIRA, C/ 444 ...",430291.0,2001-12-31,NaN
185,34468.0,JARDIM DO INFANTARIO DE CRESTINS EM MOREIRA CO...,430291.0,2001-12-31,INFANTARIO DE CRESTINS
249,118233.0,"PARQUE INFANTIL DE FREJUFE, NA RUA PADRE ANTER...",430291.0,2010-03-11,RUA PADRE ANTERO FERREIRA COSTA
250,119878.0,MONUMENTO AS COLECTIVIDADES MAIATAS - ROTUNDA ...,43023.0,2010-07-02,ROTUNDA JUNTO A ESCOLA DE MOUTIDOS
...,...,...,...,...,...
2221,149249.0,PARCELA DE TERRENOS DESTINADA A ANTIGA E.B.1 D...,4311.0,2017-12-29,RUA DA LIBERDADE - GONDIM
2328,156523.0,PARCELA DE TERRENO PARA A EB1/JI DA PROZELA - ...,4311.0,2019-11-11,RUA DA CARALINDA
2352,169380.0,PARCELA DE TERRENO DESTINADA A CONSTRUÇAO DE E...,4311.0,2021-09-07,"RUA MANUEL FRANCISCO DE ARAUJO, LUGAR DA GRANJA"
2353,169381.0,PARCELA DE TERRENO DESTINADA A CONSTRUÇAO DE E...,4311.0,2021-09-07,"RUA MANUEL FRANCISCO DE ARAUJO, LUGAR DA GRANJA"


### Filtragem da tabela "Mapa Certificação Energética"

***filtered_energetica*** : filtração da tabela ***energetica*** onde são retiradas as colunas em branco

***filtered_energetica_2*** : filtração da tabela ***filtered_energetica*** onde só sao incluídas as entradas que na coluna ***Tipologia*** sejam ***Escolares*** ou ***Santa Casa***

In [ ]:

empty_columns_energetica = energetica.columns[energetica.isnull().all()]
filtered_energetica = energetica.drop(empty_columns_energetica, axis=1)


filtered_energetica_2 = filtered_energetica[filtered_energetica["Tipologia"].isin(["Escolares", "Santa Casa"])]


filtered_energetica.head()

,Nome,Tipologia,Caracterização do Edifício,Área Útil (m2),Certificado,Data Emissão,Validade,Classe Obtida,Classe atingir,PDEE
0,TORRE LIDADOR,Diversas,Grande Edifício Comércio Serviços,11926,SCE246697539,2021-04-30,2029-04-30,C,B-,Não
1,FÓRUM MAIA,Diversas,Grande Edifício Comércio Serviços,7921.1,SCE224871529,2020-08-26,2028-08-26,C,B-,Não
2,CENTRO ESCOLAR MAIA-ESTAÇÃO,Escolares,Grande Edifício Comércio Serviços,6100.8,SCE198550735,2020-02-27,2028-02-27,D,B-,Sim
3,PAÇOS CONCELHO,Diversas,Grande Edifício Comércio Serviços,5347.9,SCE202097222,2019-08-02,2027-08-02,D,B,Sim
4,COMPLEXO MUNICIPAL TÉNIS,Desportivas,Grande Edifício Comércio Serviços,4736,SCE173308527,2018-11-12,2026-11-12,C,C,Não


### Filtragem da tabela "Tabela de Edifícios - ISOV3"

***filtered_edificios*** : filtração da tabela ***edificios*** onde são retiradas colunas desnecessárias e colunas em branco

***filtered_edificios_2*** : filtração da tabela ***filtered_edificios*** onde entradas em que a coluna ***Tipologia*** não seja ***Escolares*** são retiradas

In [ ]:
filtered_edificios = edificios.drop(columns=["Ano de construção"])
empty_columns_edificios = edificios.columns[edificios.isnull().all()]
filtered_edificios = filtered_edificios.drop(empty_columns_edificios, axis=1)

filtered_edificios_2 = filtered_edificios[filtered_edificios["Tipologia"].isin(["Escolares"])]

filtered_edificios.head()

,????,Código1,Código do Edifício,Designação,Tipologia,Morada,Localidade,Código Postal,Data da Receção,Observações,N.º Inventário,Área Coberta\n(m2?),N.º de pisos,Tipo de edifício,Tem Wifi ?,Fase Fotovoltaico,Longitude,Latitude,Gstat,URL de informação pública
0,Aeródromo,D29,125.0,Aeródromo Vilar de Luz - Aerogare,Diversas,Rua Vilar de Luz,Folgosa,NaN,NaN,NaN,NaN,2407.4,NaN,NaN,Não,Phase 3,-8.519273,41.279204,NaN,NaN
1,Aeródromo,NaN,NaN,Aeródromo Vilar de Luz - bar/restaurante,Diversas,Rua Vilar de Luz,NaN,NaN,NaN,NaN,NaN,163.3,NaN,NaN,Não,Phase 3,-8.517436,41.280707,NaN,NaN
2,Aeródromo,NaN,107.0,Aeródromo Vilar de Luz (edificios e Hangares),Diversas,NaN,NaN,NaN,NaN,NaN,143504,5069.5,2.0,NaN,Não,Phase 3,-8.516903,41.280251,NaN,NaN
3,Não definida,F3,94.0,Armazém C Civil - Junto à PSP de A. Santas,Diversas,R. Ferreira de Castro,Águas Santas,4425-083 Maia,NaN,engloba o edificio total,134444,NaN,5.0,NaN,Não,Phase 4,NaN,NaN,NaN,NaN
4,Não definida,NaN,NaN,Armazém DGOL,Diversas,NaN,NaN,NaN,NaN,engloba o edificio total,NaN,NaN,NaN,NaN,Não,Phase 4,NaN,NaN,NaN,NaN


## Recolha de dados


### Tabela Baze

In [ ]:
url = "https://baze.cm-maia.pt/BaZe/api/api4gj.php?nome=EdifCM%20-%20Escolas"

response = requests.get(url)
data = response.json()

entries = data["features"]

rows=[]
for entry in entries:
    designação = entry["properties"].get("popupContent", "")
    longitude = entry["geometry"]["coordinates"][0]
    latitude = entry["geometry"]["coordinates"][1]
    rows.append({
        "designação": designação,
        "longitude": longitude,
        "latitude": latitude
    })

baze = pd.DataFrame(rows)

baze

,designação,longitude,latitude
0,EB Bajouca,-8.636671,41.274883
1,EB Ferreiró,-8.600348,41.273765
2,EB Ferronho,-8.613663,41.278121
3,EB Castelo da Maia,-8.614067,41.259409
4,EB23 Castelo da Maia,-8.612881,41.259004
5,ES Castelo da Maia,-8.610500,41.258902
6,Escola Básica de 1.º CEB/JI de Porto Bom,-8.600269,41.255225
7,EB Frejufe,-8.587789,41.258178
8,EB Parada,-8.593115,41.200959
9,Eb Monte Calvário,-8.586010,41.236332


### Tabela Escolas_OSM

In [ ]:
# Load Maia municipal boundary
maia_url = "https://baze.cm-maia.pt/BaZe/api/api4gj.php?nome=limconcvm"
maia_gdf = gpd.read_file(maia_url)

# Convert LineString to Polygon if necessary
geom = maia_gdf.union_all()
if geom.geom_type == "LineString":
    coords = list(geom.coords)
    if coords[0] != coords[-1]:  # ensure it's closed
        coords.append(coords[0])
    maia_polygon = Polygon(coords)
else:
    maia_polygon = geom

# Expand bounding box for OSM query
minx, miny, maxx, maxy = maia_polygon.bounds
buffer = 0.01
query = f"""
[out:json][timeout:30];
(
  node["amenity"~"school|kindergarten|college|university|training"]({miny-buffer},{minx-buffer},{maxy+buffer},{maxx+buffer});
  way["amenity"~"school|kindergarten|college|university|training"]({miny-buffer},{minx-buffer},{maxy+buffer},{maxx+buffer});
  relation["amenity"~"school|kindergarten|college|university|training"]({miny-buffer},{minx-buffer},{maxy+buffer},{maxx+buffer});
);
out center tags;
"""
response = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
elements = response.json().get("elements", [])

# Parse OSM school results
schools = []
for e in elements:
    # Extract latitude and longitude
    lat, lon = e.get("lat"), e.get("lon")

    # If it's a way or relation, extract the center coordinates
    if "center" in e:
        lat, lon = e["center"]["lat"], e["center"]["lon"]

    # Ensure valid coordinates
    if lat is not None and lon is not None:
        # Extract school type from the 'amenity' or 'educational_institution' tags
        amenity = e.get("tags", {}).get("amenity", None)
        educational_institution = e.get("tags", {}).get("educational_institution", None)

        # If either 'amenity' or 'educational_institution' indicates a school, assign the school type
        if amenity in ["school", "kindergarten", "college", "university", "training"]:
            school_type = amenity
        elif educational_institution in ["school", "kindergarten", "college", "university", "training", "vocational_school"]:
            school_type = educational_institution
        else:
            school_type = "Unknown"

        # Append the school info (including the type) to the list
        schools.append({
            "Name": e.get("tags", {}).get("name", "Unnamed"),
            "Type": school_type,  # Add school type to the data
            "Latitude": lat,
            "Longitude": lon
        })

escolas_OSM = pd.DataFrame(schools)

escolas_OSM.head()

,Name,Type,Latitude,Longitude
0,Jardim de Infância da Santa Casa da Misericórdia,kindergarten,41.186491,-8.683652
1,Jardim de Infância Fonte dos Dois Amigos,kindergarten,41.186652,-8.686198
2,Infantário Pica Pau Amarelo,kindergarten,41.218561,-8.546001
3,Espaço Jota,kindergarten,41.186957,-8.645439
4,Escola de Condução Brilhante,Unknown,41.193475,-8.609859


In [ ]:
filename = 'escolas_OSM.xlsx'
escolas_OSM.to_excel(filename, index=False)

button = widgets.Button(
    description='Download escola_OSM',
    button_style='primary'
    )

def on_button_click(b):
    files.download(filename)

button.on_click(on_button_click)
print("A tabela obtida pelo OpenStreetMaps que contem as escolas da Maia: ")
display(button)

A tabela obtida pelo OpenStreetMaps que contem as escolas da Maia: 


Button(button_style='primary', description='Download escola_OSM', style=ButtonStyle())

## Matching de tabelas

### Fórmula de Haversine

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

### Matching entre a tabela ***filtered_imoveis*** e a tabela ***filtered_edificios*** através da coluna-chave ***Nº Inventário***

- Os valores de ambas as colunas foram convertidos para string e na coluna dos ***filtered_imoveis*** foi retirado aos valores a substring ".0"
- Foi usado um threshold de 100 para o score do fuzz.ratio, que calcula a distância de levenshtein, visto que à miníma diferença no nº de inventário falhava o matching
- Foram retirados os matchings de valores nulos
- A tabela de match é constituída por ambas as colunas que foram comparadas, seguidas pela designação do edifício, e por fim a coluna do score

In [ ]:
THRESHOLD = 100

matches_list = []

processed_matches = set()

filtered_imoveis['N.º INVENTARIO'] = filtered_imoveis['N.º INVENTARIO'].astype(str).str.strip()
filtered_edificios['N.º Inventário'] = filtered_edificios['N.º Inventário'].astype(str).str.strip()

filtered_imoveis['N.º INVENTARIO'] = filtered_imoveis['N.º INVENTARIO'].str.replace(r'\.0$', '', regex=True)

for _, imoveis_row in filtered_imoveis.iterrows():
    number_imoveis = imoveis_row['N.º INVENTARIO']
    designation_imoveis = imoveis_row['DESIGNAÇAO']

    matches = process.extract(number_imoveis, filtered_edificios['N.º Inventário'], scorer=fuzz.ratio, limit=None)

    for match in matches:
        if match[1] >= THRESHOLD and match[0] != 'nan':
            matching_edificios = filtered_edificios[filtered_edificios['N.º Inventário'] == match[0]]
            if number_imoveis not in processed_matches:
                for _, edificios_row in matching_edificios.iterrows():
                    matches_list.append({
                        'number_imoveis': number_imoveis,
                        'designation_imoveis': designation_imoveis,
                        'number_edificios': match[0],
                        'designation_edificios': edificios_row['Designação'],
                        'match_score': match[1]
                    })
                    processed_matches.add(match[0])

match_table = pd.DataFrame(matches_list)

match_table

,number_imoveis,designation_imoveis,number_edificios,designation_edificios,match_score
0,5814,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CAS...,5814,Mercado do Castêlo,100.0
1,6972,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,100.0
2,6973,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,100.0
3,6975,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS D...,6975,Paços do Concelho da Maia,100.0
4,24755,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS...,24755,Torre Lidador - Câmara Municipal da Maia,100.0
...,...,...,...,...,...
78,142707,EDIFICIO DESTINADO A CRECHE/INFANTARIO - VERMO...,142707,"Creche, Pré-Escolar e ATL de Vermoim",100.0
79,142740,CENTRO ESCOLAR D. MANUEL I(ANTIGA PAMAIAL)(E.B...,142740,EB1 / JI D Manuel II,100.0
80,143504,AERODROMO DE VILAR DE LUZ-FOLGOSA (REVOGADO - ...,143504,Aeródromo Vilar de Luz (edificios e Hangares),100.0
81,156697,EDIFICIO BAR DE APOIO ZONA DESPORTIVA CENTRAL ...,156697,Café/Bar Parque Maia,100.0


#### Indicadores de Matching

In [ ]:
total_imoveis = len(filtered_imoveis)
total_edificios = len(filtered_edificios)

matched_imoveis = match_table['designation_imoveis'].nunique()
matched_edificios = match_table['designation_edificios'].nunique()

percentage_imoveis_matched = (matched_imoveis / total_imoveis) * 100
percentage_edificios_matched = (matched_edificios / total_edificios) * 100

print(f"Total entries from imoveis: {total_imoveis}")
print(f"Total entries from edificios: {total_edificios}")

print(f"Matched entries from 'imoveis': {matched_imoveis} ({percentage_imoveis_matched:.2f}%)")
print(f"Matched entries from 'edificios': {matched_edificios} ({percentage_edificios_matched:.2f}%)")

Total entries from imoveis: 2449
Total entries from edificios: 177
Matched entries from 'imoveis': 82 (3.35%)
Matched entries from 'edificios': 83 (46.89%)


In [ ]:
duplicated_rows = match_table[match_table.duplicated('number_imoveis', keep=False)]

duplicated_rows

,number_imoveis,designation_imoveis,number_edificios,designation_edificios,match_score
14,46746,COMPLEXO DE TENIS DA MAIA,46746,Complexo Municipal de Ginástica da Maia,100.0
15,46746,COMPLEXO DE TENIS DA MAIA,46746,Complexo Municipal de Ténis,100.0


In [ ]:
edificios_imoveis = match_table[['number_imoveis','designation_imoveis', 'number_edificios', 'designation_edificios']]
filtered_imoveis_morada = filtered_imoveis[['DESIGNAÇAO', 'LOCALIZAÇÃO']]
filtered_edificios_morada = filtered_edificios[['Designação', 'Morada', 'Localidade', 'Código Postal', 'Longitude', 'Latitude']]

edificios_imoveis = edificios_imoveis.merge(
    filtered_imoveis_morada,
    left_on='designation_imoveis',
    right_on='DESIGNAÇAO',
    how='left'
).drop(columns=['DESIGNAÇAO'])

edificios_imoveis = edificios_imoveis.merge(
    filtered_edificios_morada,
    left_on='designation_edificios',
    right_on='Designação',
    how='left'
).drop(columns=['Designação'])

edificios_imoveis = edificios_imoveis.rename(columns ={
    'LOCALIZAÇÃO': 'localizacao_imoveis',
    'Morada': 'morada_edificios',
    'Localidade': 'localidade_edificios',
    'Código Postal': 'codigo_postal_edificios',
    'Longitude':'Lon_edificios',
    'Latitude':'Lat_edificios'
})

edificios_imoveis

,number_imoveis,designation_imoveis,number_edificios,designation_edificios,localizacao_imoveis,morada_edificios,localidade_edificios,codigo_postal_edificios,Lon_edificios,Lat_edificios
0,5814,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CAS...,5814,Mercado do Castêlo,LUGAR DO CASTELO,NaN,NaN,NaN,NaN,NaN
1,6972,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,RUA ENGº DUARTE PACHECO E RUA PADRE ANTONIO,Praça do Município,Maia,4474-006 Maia,-8.622233,41.234000
2,6973,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,RUA DA ESTAÇAO,"Rua da Estação, 35",Maia,4470-178 Maia,NaN,NaN
3,6975,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS D...,6975,Paços do Concelho da Maia,PRAÇA DO MUNICIPIO,Praça Doutor José Vieira de Carvalho,Maia,4474-006 Maia,-8.622518,41.233341
4,24755,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS...,24755,Torre Lidador - Câmara Municipal da Maia,========EDIFICIO DE PAÇOS DO CONCELHO=========...,Praça Doutor José Vieira de Carvalho,Maia,4474-006 Maia,-8.622039,41.233396
...,...,...,...,...,...,...,...,...,...,...
78,142707,EDIFICIO DESTINADO A CRECHE/INFANTARIO - VERMO...,142707,"Creche, Pré-Escolar e ATL de Vermoim",RUA CENTRAL DO SOBREIRO - VERMOIM,"Rua Central do Sobreiro, 79",Vermoim,NaN,NaN,NaN
79,142740,CENTRO ESCOLAR D. MANUEL I(ANTIGA PAMAIAL)(E.B...,142740,EB1 / JI D Manuel II,Escola E.B.1/J.I.D.Manuel I,Rua do Estádio,Vermoim,4470-371 Maia,-8.617173,41.235816
80,143504,AERODROMO DE VILAR DE LUZ-FOLGOSA (REVOGADO - ...,143504,Aeródromo Vilar de Luz (edificios e Hangares),VILAR DE LUZ - FOLGOSA,NaN,NaN,NaN,-8.516903,41.280251
81,156697,EDIFICIO BAR DE APOIO ZONA DESPORTIVA CENTRAL ...,156697,Café/Bar Parque Maia,ZONA DESPORTIVA DA MAIA,NaN,NaN,NaN,NaN,NaN


### Matching das tabelas ***edificios_imoveis*** e ***filtered_contadores***


Preparação do matching:
 - foram retiradas as entradas que tinham valores nulos na coluna de código postal na tabela ***edificios_imoveis***
 - Em ambas as tabelas foram usados apenas os primeiros 8 caracteres pois na tabela ***edificios_imoveis*** continha sempre a palavra Maia depois do código postal, e na tabela ***filtered_contadores*** continha espaços desnecessários

In [ ]:
edificios_imoveis_cod = edificios_imoveis.dropna(subset=['codigo_postal_edificios'])

edificios_imoveis_cod.loc[:, 'codigo_postal_edificios'] = edificios_imoveis_cod['codigo_postal_edificios'].str[:8]

filtered_contadores['cod_fpost'] = filtered_contadores['cod_fpost'].str[:8]


In [ ]:
THRESHOLD = 100

matches_list= []
processed_matches = set()

for _, edificios_imoveis_row in edificios_imoveis_cod.iterrows():
    cod_edificios_imoveis = edificios_imoveis_row['codigo_postal_edificios']
    designation_imoveis = edificios_imoveis_row['designation_imoveis']
    designation_edificios = edificios_imoveis_row['designation_edificios']
    lat_edificios = edificios_imoveis_row['Lat_edificios']
    lon_edificios = edificios_imoveis_row['Lon_edificios']
    number = edificios_imoveis_row['number_imoveis']

    matches = process.extract(cod_edificios_imoveis, filtered_contadores['cod_fpost'], scorer=fuzz.ratio, limit=None)

    for match in matches:
        if match[1] >= THRESHOLD and match[0] != 'nan':
            matching_contadores = filtered_contadores[filtered_contadores['cod_fpost'] == match[0]]

            if cod_edificios_imoveis not in processed_matches:
                for _, contadores_row in matching_contadores.iterrows():
                    matches_list.append({
                        'number_inventario': number,
                        'cod_edificios': cod_edificios_imoveis,
                        'designation_edificios': designation_edificios,
                        'designation_imoveis': designation_imoveis,
                        'cod_contadores': match[0],
                        'designation_contadores': contadores_row['desig_edificio'],
                        'lat_edificios': lat_edificios,
                        'lon_edificios': lon_edificios,
                        'lat_contadores': contadores_row['Lat'],
                        'lon_contadores': contadores_row['Lon'],
                        'match_score_cod': match[1]
                    })
                    processed_matches.add(match[0])

imo_edi_con = pd.DataFrame(matches_list)

imo_edi_con

,number_inventario,cod_edificios,designation_edificios,designation_imoveis,cod_contadores,designation_contadores,lat_edificios,lon_edificios,lat_contadores,lon_contadores,match_score_cod
0,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,Oficinas Gerais,NaN,NaN,NaN,NaN,100.0
1,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,NaN,NaN,NaN,NaN,NaN,100.0
2,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,Rega,NaN,NaN,NaN,NaN,100.0
3,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,NaN,NaN,NaN,NaN,NaN,100.0
4,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,Oficinas,NaN,NaN,NaN,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...
75,141603,4425-186,Centro Escolar de Folgosa,CENTRO ESCOLAR DE FOLGOSA (COMODATO DE PARTE D...,4425-186,Centro Escolar Folgosa,41.260326,-8.547467,NaN,NaN,100.0
76,141603,4425-186,Centro Escolar de Folgosa,CENTRO ESCOLAR DE FOLGOSA (COMODATO DE PARTE D...,4425-186,Centro Escolar Folgosa,41.260326,-8.547467,NaN,NaN,100.0
77,141603,4425-186,Centro Escolar de Folgosa,CENTRO ESCOLAR DE FOLGOSA (COMODATO DE PARTE D...,4425-186,Centro Escolar Folgosa,41.260326,-8.547467,NaN,NaN,100.0
78,142082,4475-054,Centro Escolar de Mandim,CENTRO ESCOLAR DE MANDIM,4475-054,Centro Escolar,NaN,NaN,NaN,NaN,100.0


Depois de uma análise feita à tabela, verifica-se que as colunas ***lat_contadores*** e ***lon_contadores*** apenas contêm valores nulos podendo então ser removidas

In [ ]:
imo_edi_con = imo_edi_con.drop(columns=['lat_contadores', 'lon_contadores'])

In [ ]:
styled_imo_edi_con = imo_edi_con.style \
                .map(lambda val: 'background-color: beige', subset=['number_inventario']) \
                .map(lambda val: 'background-color: lightblue', subset=['designation_imoveis']) \
                .map(lambda val: 'background-color: lightgreen', subset=['cod_contadores', 'designation_contadores']) \
                .map(lambda val: 'background-color: lightcoral', subset=['cod_edificios', 'designation_edificios', 'lat_edificios', 'lon_edificios']) \
                .map(lambda val: 'background-color: yellow', subset=['match_score_cod']) \

styled_imo_edi_con

,number_inventario,cod_edificios,designation_edificios,designation_imoveis,cod_contadores,designation_contadores,lat_edificios,lon_edificios,match_score_cod
0,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,Oficinas Gerais,nan,nan,100.000000
1,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,nan,nan,nan,100.000000
2,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,Rega,nan,nan,100.000000
3,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,nan,nan,nan,100.000000
4,6973,4470-178,Oficinas Gerais da Maia,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,4470-178,Oficinas,nan,nan,100.000000
5,38754,4470-030,Complexo Municipal de Piscinas de Gueifães,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),4470-030,Rega,41.214994,-8.600761,100.000000
6,38754,4470-030,Complexo Municipal de Piscinas de Gueifães,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),4470-030,Compl Municipal Piscinas,41.214994,-8.600761,100.000000
7,46680,4425-144,Polidesportivo Municipal dos Moutidos,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",4425-144,Rega,nan,nan,100.000000
8,46687,4425-053,Pavilhão Municipal de Águas Santas 1 - Ardegães,"PAVILHAO MUNICIPAL DE ARDEGAES (PAVILHAO MUNICIPAL DE AGUAS SANTAS ""4"" (I) - ARDEGAES)",4425-053,nan,nan,nan,100.000000
9,46687,4425-053,Pavilhão Municipal de Águas Santas 1 - Ardegães,"PAVILHAO MUNICIPAL DE ARDEGAES (PAVILHAO MUNICIPAL DE AGUAS SANTAS ""4"" (I) - ARDEGAES)",4425-053,nan,nan,nan,100.000000


### Matching das tabelas ***edificios_imoveis*** e ***filtered_contadores_2***

In [ ]:
edificios_imoveis_coord = edificios_imoveis[['designation_imoveis', 'number_edificios', 'designation_edificios', 'Lon_edificios', 'Lat_edificios']]
contadores_2_coord = filtered_contadores_2[['Lat', 'Lon', 'desig_edificio', 'Nome da Escola']]

In [ ]:
edificios_imoveis_coord['key'] = 1
contadores_2_coord['key'] = 1
cross = pd.merge(edificios_imoveis_coord, contadores_2_coord, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Lat_edificios'], row['Lon_edificios'], row['Lon'], row['Lat']),
    axis=1
)

distance_threshold = 100
imo_edi_con_2 = cross[cross['distance_m'] <= distance_threshold]

imo_edi_con_2 = imo_edi_con_2.rename(columns={
    'number_edificios':'Nº inventario',
    'Lon':'Lat_contadores_2',
    'Lat':'Lon_contadores_2',
    'desig_edificio':'designação_contadores_2',
    'Nome da Escola':'Escola_contadores_2'
})

imo_edi_con_2

/tmp/ipython-input-23-1437177526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edificios_imoveis_coord['key'] = 1
/tmp/ipython-input-23-1437177526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contadores_2_coord['key'] = 1


,designation_imoveis,Nº inventario,designation_edificios,Lon_edificios,Lat_edificios,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,distance_m
1698,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,-8.622233,41.234000,-8.623223,41.234393,NaN,NaN,93.600387
3464,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS D...,6975,Paços do Concelho da Maia,-8.622518,41.233341,-8.622487,41.233012,NaN,NaN,36.730010
4360,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS...,24755,Torre Lidador - Câmara Municipal da Maia,-8.622039,41.233396,-8.622487,41.233012,NaN,NaN,56.790272
4495,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICI...,38754,Complexo Municipal de Piscinas de Gueifães,-8.600761,41.214994,-8.599861,41.215541,Rega,NaN,96.794696
5300,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICI...,38754,Complexo Municipal de Piscinas de Gueifães,-8.600761,41.214994,-8.600243,41.214185,NaN,EB de Gueifães,99.855046
5302,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICI...,38754,Complexo Municipal de Piscinas de Gueifães,-8.600761,41.214994,-8.600243,41.214185,NaN,EB de Gueifães,99.855046
5303,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICI...,38754,Complexo Municipal de Piscinas de Gueifães,-8.600761,41.214994,-8.600243,41.214185,NaN,EB de Gueifães,99.855046
12620,COMPLEXO DE TENIS DA MAIA,46746,Complexo Municipal de Ginástica da Maia,-8.615070,41.233827,-8.614649,41.234342,Rega,NaN,67.151877
13285,COMPLEXO DE TENIS DA MAIA,46746,Complexo Municipal de Ginástica da Maia,-8.615070,41.233827,-8.613909,41.233700,Rega-Rotunda,NaN,98.086049
13691,COMPLEXO DE TENIS DA MAIA,46746,Complexo Municipal de Ténis,-8.615797,41.233634,-8.616547,41.233600,NaN,NaN,62.871815


In [ ]:
print(len(imo_edi_con_2))
print(len(imo_edi_con))

48
80


### Matching das tabelas ***imo_edi_con*** e ***baze***

Preparação da tabela para o matching com a tabela ***baze***

In [ ]:
imo_edi_con_cd = imo_edi_con.drop(['cod_contadores', 'match_score_cod'], axis=1)

imo_edi_con_cd = imo_edi_con_cd.dropna(subset=['lat_edificios'])

imo_edi_con_cd

,number_inventario,cod_edificios,designation_edificios,designation_imoveis,designation_contadores,lat_edificios,lon_edificios
5,38754,4470-030,Complexo Municipal de Piscinas de Gueifães,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICI...,Rega,41.214994,-8.600761
6,38754,4470-030,Complexo Municipal de Piscinas de Gueifães,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICI...,Compl Municipal Piscinas,41.214994,-8.600761
54,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027
55,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027
56,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027
57,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Urb Real Castelo-Rega,41.259439,-8.614027
58,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027
59,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Centro Esc Gueifaes/Vermoim,41.226438,-8.608122
60,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Centro Esc Gueifaes/Vermoim,41.226438,-8.608122
61,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Centro Esc Gueifaes/Vermoim,41.226438,-8.608122


Matching da tabela ***imo_edi_con_cd*** com a tabela ***baze*** feito com um cross join e depois filtrado com um distance_threshold de 100 metros

In [ ]:
imo_edi_con_cd['key'] = 1
baze['key'] = 1
cross = pd.merge(imo_edi_con_cd, baze, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['lat_edificios'], row['lon_edificios'], row['latitude'], row['longitude']),
    axis=1
)

distance_threshold = 50
imo_edi_con_baze = cross[cross['distance_m'] <= distance_threshold]

imo_edi_con_baze = imo_edi_con_baze.rename(columns={
    'designação':'designation_baze',
    'longitude':'lon_baze',
    'latitude':'lat_baze'
})

imo_edi_con_baze

,number_inventario,cod_edificios,designation_edificios,designation_imoveis,designation_contadores,lat_edificios,lon_edificios,designation_baze,lon_baze,lat_baze,distance_m
97,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027,EB Castelo da Maia,-8.614067,41.259409,4.718988
144,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027,EB Castelo da Maia,-8.614067,41.259409,4.718988
191,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027,EB Castelo da Maia,-8.614067,41.259409,4.718988
238,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Urb Real Castelo-Rega,41.259439,-8.614027,EB Castelo da Maia,-8.614067,41.259409,4.718988
285,100043,4475-615,EB1 Castêlo da Maia,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),Rega,41.259439,-8.614027,EB Castelo da Maia,-8.614067,41.259409,4.718988
362,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Centro Esc Gueifaes/Vermoim,41.226438,-8.608122,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678
409,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Centro Esc Gueifaes/Vermoim,41.226438,-8.608122,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678
456,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Centro Esc Gueifaes/Vermoim,41.226438,-8.608122,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678
503,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,Rega,41.226438,-8.608122,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678
550,130783,4470-795,Centro Escolar de Vermoim/Gueifães,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,NaN,41.226438,-8.608122,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678


### Matching das tabelas ***edificios_imoveis*** e ***baze***

In [ ]:
edificios_imoveis_coord['key'] = 1
baze['key'] = 1
cross = pd.merge(edificios_imoveis_coord, baze, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Lat_edificios'], row['Lon_edificios'], row['latitude'], row['longitude']),
    axis=1
)

distance_threshold = 100
imo_edi_baze = cross[cross['distance_m'] <= distance_threshold]

imo_edi_baze = imo_edi_baze.rename(columns={
    'number_edificios':'Nº inventario',
    'latitude':'lat_baze',
    'longitude':'lon_baze',
    'designation':'designation_baze'
})

imo_edi_baze

/tmp/ipython-input-27-657096221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edificios_imoveis_coord['key'] = 1


,designation_imoveis,Nº inventario,designation_edificios,Lon_edificios,Lat_edificios,designação,lon_baze,lat_baze,distance_m
729,COMPLEXO DE TENIS DA MAIA,46746,Complexo Municipal de Ténis,-8.615797,41.233634,EB23 Gonçalo Mendes da Maia,-8.615349,41.232819,98.081121
2088,E.B.1/J.I. DE PEDROUÇOS (PEDROUÇOS),80152,EB1/JI Pedrouços,-8.589581,41.183256,EB Pedrouços,-8.589668,41.183069,21.982533
2837,E.B.1/J.I. DA MAIA (MAIA SEDE),100042,EB1 da Maia - (Maia Sede - Via Periférica),-8.619372,41.226900,EB Maia,-8.619240,41.226872,11.531264
2870,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),100043,EB1 Castêlo da Maia,-8.614027,41.259439,EB Castelo da Maia,-8.614067,41.259409,4.718988
3041,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,130783,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678
3134,"NOVO CENTRO ESCOLAR DA MAIA (ESTAÇAO EB1/JI), ...",132345,Centro Escolar da Maia,-8.633818,41.234086,Centro escolar Maia Estação,-8.633343,41.234084,39.703538
3426,E.B.1 DA PICUA - CENTRO ESCOLAR QUINTA DA PICU...,138124,EB1 Pícua,-8.568400,41.204350,EB Pícua,-8.568378,41.204116,26.060128
3513,CENTRO ESCOLAR DE FOLGOSA (COMODATO DE PARTE D...,141603,Centro Escolar de Folgosa,-8.547467,41.260326,Centro Escolar Folgosa,-8.547591,41.260314,10.448034


### Matching das tabelas ***edificios_imoveis*** e ***escolas_OSM***

In [ ]:
edificios_imoveis_coord['key'] = 1
escolas_OSM['key'] = 1
cross = pd.merge(edificios_imoveis_coord, escolas_OSM, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Lat_edificios'], row['Lon_edificios'], row['Latitude'], row['Longitude']),
    axis=1
)

distance_threshold = 100
imo_edi_escolas_OSM = cross[cross['distance_m'] <= distance_threshold]

imo_edi_escolas_OSM = imo_edi_escolas_OSM.rename(columns={
    'number_edificios':'Nº inventario',
    'Latitude':'lat_escolas_OSM',
    'Longitude':'lon_escolas_OSM',
    'designation':'designation_escolas_OSM'
})

imo_edi_escolas_OSM

/tmp/ipython-input-28-3840798765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edificios_imoveis_coord['key'] = 1


,designation_imoveis,Nº inventario,designation_edificios,Lon_edificios,Lat_edificios,Name,Type,lat_escolas_OSM,lon_escolas_OSM,distance_m
16733,E.B.1/J.I. DE PEDROUÇOS (PEDROUÇOS),80152,EB1/JI Pedrouços,-8.589581,41.183256,Escola Básica n.º 2 de Pedrouços,school,41.183170,-8.589583,9.484509
22729,E.B.1/J.I. DA MAIA (MAIA SEDE),100042,EB1 da Maia - (Maia Sede - Via Periférica),-8.619372,41.226900,Escola Básica da Maia,school,41.226554,-8.619460,39.247463
22989,E.B.1/J.I. DA MAIA (MAIA SEDE),100042,EB1 da Maia - (Maia Sede - Via Periférica),-8.619372,41.226900,Jardim de Infância da Maia,kindergarten,41.227635,-8.619374,81.661650
23150,E.B.1/J.I. DO CASTELO DA MAIA (AVIOSO STª MARIA),100043,EB1 Castêlo da Maia,-8.614027,41.259439,Escola Básica do 1º Ciclo e Jardim de Infância...,school,41.259417,-8.613878,12.723951
24240,CENTRO ESCOLAR VERMOIM-GUEIFAES - EB1 E JI COM...,130783,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,Escola Básica n.º 1 de Gueifães,school,41.226159,-8.607879,37.082108
25015,"NOVO CENTRO ESCOLAR DA MAIA (ESTAÇAO EB1/JI), ...",132345,Centro Escolar da Maia,-8.633818,41.234086,Escola Básica do 1º Ciclo e Jardim de Infância...,school,41.234299,-8.633845,23.803753
27243,E.B.1 DA PICUA - CENTRO ESCOLAR QUINTA DA PICU...,138124,EB1 Pícua,-8.568400,41.204350,Escola Básica do 1º Ciclo e Jardim de Infância...,school,41.204380,-8.568072,27.685087
28173,CENTRO ESCOLAR DE FOLGOSA (COMODATO DE PARTE D...,141603,Centro Escolar de Folgosa,-8.547467,41.260326,Centro Escolar de Folgosa,school,41.260242,-8.547480,9.443422
30060,CENTRO ESCOLAR D. MANUEL I(ANTIGA PAMAIAL)(E.B...,142740,EB1 / JI D Manuel II,-8.617173,41.235816,Escola Básica Dom Manuel II,school,41.235752,-8.617437,23.164598
30222,AERODROMO DE VILAR DE LUZ-FOLGOSA (REVOGADO - ...,143504,Aeródromo Vilar de Luz (edificios e Hangares),-8.516903,41.280251,Nortávia,Unknown,41.280335,-8.516921,9.498969


### Matching das tabelas ***filtered_edificios*** e ***baze***

Preparação da tabela edificios para o matching com a tabela ***baze***:
 - Utilização das colunas necessárias
 - Remover entradas com valor nulo na Latitude e Longitude


In [ ]:
edificios_coord = filtered_edificios[['Designação', 'Longitude', 'Latitude']]

edificios_coord = edificios_coord.dropna(subset=['Latitude'])

edificios_coord

,Designação,Longitude,Latitude
0,Aeródromo Vilar de Luz - Aerogare,-8.519273,41.279204
1,Aeródromo Vilar de Luz - bar/restaurante,-8.517436,41.280707
2,Aeródromo Vilar de Luz (edificios e Hangares),-8.516903,41.280251
8,Bairro do Sobreiro (Restante),-8.619701,41.236487
12,Blocos 41-42,-8.621800,41.236800
13,Blocos 43-47,-8.621000,41.236600
18,Casa do Alto,-8.592280,41.189443
19,Casa do Cantoneiro,-8.622239,41.236815
20,Casa do Corim,-8.575657,41.197660
25,Centro Escolar da Maia,-8.633818,41.234086


Matching da tabela de edificios com a tabela ***baze***

In [ ]:
edificios_coord['key'] = 1
baze['key'] = 1
cross = pd.merge(edificios_coord, baze, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']),
    axis=1
)

distance_threshold = 100
edificios_baze = cross[cross['distance_m'] <= distance_threshold]

edificios_baze = edificios_baze.rename(columns={
    'Designação':'designation_edificios',
    'Longitude':'lon_edificios',
    'Latitude':'lat_edificios',
    'designação':'designation_baze',
    'longitude':'lon_baze',
    'latitude':'lat_baze'
})

edificios_baze

,designation_edificios,lon_edificios,lat_edificios,designation_baze,lon_baze,lat_baze,distance_m
455,Centro Escolar da Maia,-8.633818,41.234086,Centro escolar Maia Estação,-8.633343,41.234084,39.703538
505,Centro Escolar de Folgosa,-8.547467,41.260326,Centro Escolar Folgosa,-8.547591,41.260314,10.448034
550,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,27.455678
823,Complexo Municipal de Ténis,-8.615797,41.233634,EB23 Gonçalo Mendes da Maia,-8.615349,41.232819,98.081121
850,EB Castêlo,-8.612648,41.258767,EB23 Castelo da Maia,-8.612881,41.259004,32.776642
943,EB1 Castêlo da Maia,-8.614027,41.259439,EB Castelo da Maia,-8.614067,41.259409,4.718988
1004,EB1 da Maia - (Maia Sede - Via Periférica),-8.619372,41.226900,EB Maia,-8.619240,41.226872,11.531264
1075,EB1 de Moutidos,-8.570914,41.211138,EB Moutidos,-8.570920,41.211187,5.482207
1123,EB1 Pícua,-8.568400,41.204350,EB Pícua,-8.568378,41.204116,26.060128
1148,EB1/JI Pedrouços,-8.589581,41.183256,EB Pedrouços,-8.589668,41.183069,21.982533


### Matching das tabelas ***filtered_edificios*** e ***filtered_contadores_2***

In [ ]:
edificios_coord['key'] = 1
contadores_2_coord['key'] = 1
cross = pd.merge(edificios_coord, contadores_2_coord, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], row['Lon'], row['Lat']),
    axis=1
)

distance_threshold = 100
edificios_contadores_2 = cross[cross['distance_m'] <= distance_threshold]

edificios_contadores_2 = edificios_contadores_2.rename(columns={
    'Nome da Escola':'designation_contadores_2',
    'Lat':'lon_contadores_2',
    'Lon':'lat_contadores_2',
    'Designação': 'designation_edificios',
    'Longitude':'lon_edificios',
    'Latitude':'lat_edificios',
})
edificios_contadores_2 = edificios_contadores_2.dropna(subset=['designation_contadores_2'])


display(HTML(edificios_contadores_2.to_html()))

/tmp/ipython-input-31-1419643397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contadores_2_coord['key'] = 1


,designation_edificios,lon_edificios,lat_edificios,lon_contadores_2,lat_contadores_2,desig_edificio,designation_contadores_2,distance_m
8699,Centro Escolar da Maia,-8.633818,41.234086,-8.633294,41.233894,Centro Escolar Maia,EB n.º 1 da Maia,48.707413
8707,Centro Escolar da Maia,-8.633818,41.234086,-8.633294,41.233870,Centro Escolar Maia,EB n.º 1 da Maia,49.932422
9673,Centro Escolar de Folgosa,-8.547467,41.260326,-8.547259,41.260558,Centro Escolar Folgosa,EB de Folgosa,31.039637
9674,Centro Escolar de Folgosa,-8.547467,41.260326,-8.547259,41.260558,Centro Escolar Folgosa,EB de Folgosa,31.039637
9675,Centro Escolar de Folgosa,-8.547467,41.260326,-8.547259,41.260558,Centro Escolar Folgosa,EB de Folgosa,31.039637
10495,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,-8.608094,41.226015,Centro Esc Gueifaes/Vermoim,EB n.º 1 de Gueifães,47.074541
10496,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,-8.608094,41.226015,Centro Esc Gueifaes/Vermoim,EB n.º 1 de Gueifães,47.074541
10497,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,-8.608094,41.226015,Centro Esc Gueifaes/Vermoim,EB n.º 1 de Gueifães,47.074541
12147,Complexo Municipal de Piscinas de Águas Santas,-8.564285,41.204274,-8.564852,41.203734,Escola EB1 Picua,EB de Pícua,76.585688
15156,Complexo Municipal de Piscinas de Gueifães,-8.600761,41.214994,-8.600243,41.214185,NaN,EB de Gueifães,99.855046


### Matching das tabelas ***filtered_edificios*** e ***filtered_escolas_OSM***

In [ ]:
edificios_coord = edificios_coord.rename(columns={
    'Latitude':'lat_edificios',
    'Longitude':'lon_edificios',
    'Designação': 'designation_edificios'
})


In [ ]:
edificios_coord['key'] = 1
escolas_OSM['key'] = 1
cross = pd.merge(edificios_coord, escolas_OSM, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['lat_edificios'], row['lon_edificios'], row['Latitude'], row['Longitude']),
    axis=1
)

distance_threshold = 100
edificios_escolas_OSM = cross[cross['distance_m'] <= distance_threshold]

edificios_escolas_OSM = edificios_escolas_OSM.rename(columns={
    'Name': 'designation_escolas_OSM',
    'Longitude':'lon_escolas_OSM',
    'Latitude':'lat_escolas_OSM'
})

display(HTML(edificios_escolas_OSM.to_html()))

,designation_edificios,lon_edificios,lat_edificios,designation_escolas_OSM,Type,lat_escolas_OSM,lon_escolas_OSM,distance_m
439,Aeródromo Vilar de Luz - bar/restaurante,-8.517436,41.280707,Nortávia,Unknown,41.280335,-8.516921,59.714396
816,Aeródromo Vilar de Luz (edificios e Hangares),-8.516903,41.280251,Nortávia,Unknown,41.280335,-8.516921,9.498969
2423,Casa do Alto,-8.592280,41.189443,Creche Pré-Escolar de Pedrouços,kindergarten,41.189121,-8.591818,52.653148
3178,Casa do Corim,-8.575657,41.197660,O Amanhã da Criança - Infantário,kindergarten,41.196839,-8.575456,92.802223
3526,Centro Escolar da Maia,-8.633818,41.234086,Escola Básica do 1º Ciclo e Jardim de Infância n.º 1 da Maia,school,41.234299,-8.633845,23.803753
4045,Centro Escolar de Folgosa,-8.547467,41.260326,Centro Escolar de Folgosa,school,41.260242,-8.547480,9.443422
4259,Centro Escolar de Vermoim/Gueifães,-8.608122,41.226438,Escola Básica n.º 1 de Gueifães,school,41.226159,-8.607879,37.082108
6965,EB Castêlo,-8.612648,41.258767,Escola Básica do 2º e 3º Ciclos do Castêlo da Maia,school,41.259259,-8.612484,56.386279
7440,EB1 / JI D Manuel II,-8.617173,41.235816,Escola Básica Dom Manuel II,school,41.235752,-8.617437,23.164598
7693,EB1 Castêlo da Maia,-8.614027,41.259439,Escola Básica do 1º Ciclo e Jardim de Infância do Castêlo da Maia,school,41.259417,-8.613878,12.723951


## Matching das tabelas baze, filtered_contadores_2 e escolas_OSM

In [ ]:
distance_threshold = 70

### Matching das tabelas ***filtered_contadores_2*** e ***baze*** e no fim com ***escolas_OSM***

In [ ]:
contadores_2_coord['key'] = 1
baze['key'] = 1
cross = pd.merge(contadores_2_coord, baze, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Lon'], row['Lat'], row['latitude'], row['longitude']),
    axis=1
)

contadores_baze_2 = cross[cross['distance_m'] <= distance_threshold]

contadores_baze_2 = contadores_baze_2.rename(columns={
    'Lon':'Lat_contadores_2',
    'Lat':'Lon_contadores_2',
    'desig_edificio':'designação_contadores_2',
    'Nome da Escola':'Escola_contadores_2',
    'longitude': 'Lon_baze',
    'latitude':'Lat_baze',
    'designation': 'designação_baze'
})

contadores_baze_2.head()

/tmp/ipython-input-35-226740040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contadores_2_coord['key'] = 1


,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,designação,Lon_baze,Lat_baze,distance_m
251,-8.658449,41.234673,Escola EB1/JI Crestins,EB de Crestins,Eb Crestins,-8.658670,41.234562,22.170356
755,-8.614466,41.259500,Escola EB1/JI Castelo Maia,"EB do Castelo da Maia, Santa Maria De Avioso",EB Castelo da Maia,-8.614067,41.259409,34.847964
1057,-8.614856,41.229920,NaN,NaN,EB Cidade Jardim,-8.614799,41.229635,32.028804
2323,-8.590447,41.183175,Fonte Luminosa Rotunda G M,NaN,EB Pedrouços,-8.589668,41.183069,66.271312
5901,-8.574324,41.204180,Pavilhao Municipal Aguas Santa,NaN,EBS Águas Santas,-8.574054,41.204271,24.758865


In [ ]:
escolas_OSM_matching = escolas_OSM[['Name', 'Latitude', 'Longitude']]

In [ ]:
contadores_baze_2['key'] = 1
escolas_OSM_matching['key'] = 1
cross = pd.merge(contadores_baze_2, escolas_OSM_matching, on='key').drop('key', axis=1)

cross['distance_m_2'] = cross.apply(
    lambda row: min(haversine(row['Lat_contadores_2'], row['Lon_contadores_2'], row['Latitude'], row['Longitude']), haversine(row['Lat_baze'], row['Lon_baze'], row['Latitude'], row['Longitude'])),
    axis=1
)

contadores_baze_escolas_OSM_2 = cross[cross['distance_m_2'] <= distance_threshold]

contadores_baze_escolas_OSM_2 = contadores_baze_escolas_OSM_2.rename(columns={
    'Name':'Name_escolas_OSM',
    'Latitude':'Latitude_escolas_OSM',
    'Longitude':'Longitude_escolas_OSM'
})

contadores_baze_escolas_OSM_2.head()

/tmp/ipython-input-37-2008383859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escolas_OSM_matching['key'] = 1


,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,designação,Lon_baze,Lat_baze,distance_m,Name_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM,distance_m_2
129,-8.658449,41.234673,Escola EB1/JI Crestins,EB de Crestins,Eb Crestins,-8.658670,41.234562,22.170356,Creche/Pré-Escolar de Crestins,41.234286,-8.659379,66.837473
271,-8.658449,41.234673,Escola EB1/JI Crestins,EB de Crestins,Eb Crestins,-8.658670,41.234562,22.170356,Escola Básica do 1º Ciclo de Crestins,41.234412,-8.658729,17.463567
530,-8.614466,41.259500,Escola EB1/JI Castelo Maia,"EB do Castelo da Maia, Santa Maria De Avioso",EB Castelo da Maia,-8.614067,41.259409,34.847964,Escola Básica do 1º Ciclo e Jardim de Infância...,41.259417,-8.613878,15.868954
844,-8.614856,41.229920,NaN,NaN,EB Cidade Jardim,-8.614799,41.229635,32.028804,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,21.963135
1276,-8.590447,41.183175,Fonte Luminosa Rotunda G M,NaN,EB Pedrouços,-8.589668,41.183069,66.271312,Escola Básica n.º 2 de Pedrouços,41.183170,-8.589583,13.308781


### Matching das tabelas ***filtered_contadores_2*** e ***escolas_OSM*** e no fim com ***baze***

In [ ]:
escolas_OSM_matching['key'] = 1
contadores_2_coord['key'] = 1
cross = pd.merge(escolas_OSM_matching, contadores_2_coord, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], row['Lon'], row['Lat']),
    axis=1
)

contadores_escolas_OSM_2 = cross[cross['distance_m'] <= distance_threshold]

contadores_escolas_OSM_2 = contadores_escolas_OSM_2.rename(columns={
    'Name':'Name_escolas_OSM',
    'Latitude':'Lat_escolas_OSM',
    'Longitude':'Lon_escolas_OSM',
    'Lon':'Lat_contadores_2',
    'Lat':'Lon_contadores_2',
    'desig_edificio':'designação_contadores_2',
    'Nome da Escola':'Escola_contadores_2'
})
contadores_escolas_OSM_2.head()

/tmp/ipython-input-38-3135261766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escolas_OSM_matching['key'] = 1
/tmp/ipython-input-38-3135261766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contadores_2_coord['key'] = 1


,Name_escolas_OSM,Lat_escolas_OSM,Lon_escolas_OSM,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,distance_m
7447,Centro de estudo,41.260262,-8.615592,-8.616255,41.260140,Rega,NaN,57.060550
10066,Creche Pré-Escolar de Santa Maria de Avioso,41.266698,-8.604961,-8.604253,41.266594,Rega,NaN,60.285149
16481,Creche e Pré-Escolar de Gondim,41.262391,-8.598610,-8.599217,41.262669,Rega,NaN,59.369792
16566,Creche e Pré-Escolar de Gondim,41.262391,-8.598610,-8.599174,41.262702,Rega,NaN,58.399103
25121,Escola de Música 7 Notas,41.252489,-8.662767,-8.662468,41.252396,Polidesportivo Campo Preto,NaN,27.053870


In [ ]:
contadores_escolas_OSM_2['key'] = 1
baze['key'] = 1
cross = pd.merge(contadores_escolas_OSM_2, baze, on='key').drop('key', axis=1)

cross['distance_m_2'] = cross.apply(
    lambda row: min(haversine(row['Lat_escolas_OSM'], row['Lon_escolas_OSM'], row['latitude'], row['longitude']), haversine(row['Lat_contadores_2'], row['Lon_contadores_2'], row['latitude'], row['longitude'])),
    axis=1
)

contadores_escola_OSM_baze_2 = cross[cross['distance_m_2'] <= distance_threshold]

contadores_escola_OSM_baze_2 = contadores_escola_OSM_baze_2.rename(columns={
    'longitude': 'Longitude_baze',
    'latitude':'Latitude_baze',
    'designação': 'designação_baze'
})

contadores_escola_OSM_baze_2

,Name_escolas_OSM,Lat_escolas_OSM,Lon_escolas_OSM,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,distance_m,designação_baze,Longitude_baze,Latitude_baze,distance_m_2
446,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,-8.614856,41.229920,NaN,NaN,48.147922,EB Cidade Jardim,-8.614799,41.229635,21.963135
493,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,-8.614838,41.230097,Rega,NaN,67.728254,EB Cidade Jardim,-8.614799,41.229635,21.963135
540,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,-8.614863,41.230080,Rega-Jardim Capela,NaN,65.493747,EB Cidade Jardim,-8.614799,41.229635,21.963135
587,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,-8.614928,41.230040,Escola EB1/JI Cidade Jardim,EB de Cidade Jardim,60.413161,EB Cidade Jardim,-8.614799,41.229635,21.963135
652,Escola Básica de Moutidos,41.211528,-8.570596,-8.570950,41.211343,Rega,EB de Moutidos,36.093509,EB Moutidos,-8.570920,41.211187,17.521407
699,Escola Básica de Moutidos,41.211528,-8.570596,-8.570950,41.211343,NaN,EB de Moutidos,36.093509,EB Moutidos,-8.570920,41.211187,17.521407
746,Escola Básica de Moutidos,41.211528,-8.570596,-8.570950,41.211343,NaN,EB de Moutidos,36.093509,EB Moutidos,-8.570920,41.211187,17.521407
795,Centro Escolar da Gandra,41.216359,-8.561883,-8.562471,41.216511,Escola Gandra,EB de Gandra,51.966741,EB Gandra,-8.562116,41.216358,19.422158
842,Centro Escolar da Gandra,41.216359,-8.561883,-8.562471,41.216511,Escola Gandra,EB de Gandra,51.966741,EB Gandra,-8.562116,41.216358,19.422158
889,Centro Escolar da Gandra,41.216359,-8.561883,-8.562471,41.216511,Escola Gandra,EB de Gandra,51.966741,EB Gandra,-8.562116,41.216358,19.422158


### Matching das tabelas ***escolas_OSM*** e ***baze*** e no fim com ***filtered_contadores_2***

In [ ]:
escolas_OSM_matching['key'] = 1
baze['key'] = 1
cross = pd.merge(escolas_OSM_matching, baze, on='key').drop('key', axis=1)

cross['distance_m'] = cross.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']),
    axis=1
)

baze_escolas_OSM = cross[cross['distance_m'] <= distance_threshold]

baze_escolas_OSM = baze_escolas_OSM.rename(columns={
    'Name':'Name_escolas_OSM',
    'Latitude':'Lat_escolas_OSM',
    'Longitude':'Lon_escolas_OSM',
    'longitude': 'Lon_baze',
    'latitude':'Lat_baze',
    'designation': 'designação_baze'
})

baze_escolas_OSM

/tmp/ipython-input-40-3255685259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escolas_OSM_matching['key'] = 1


,Name_escolas_OSM,Lat_escolas_OSM,Lon_escolas_OSM,designação,Lon_baze,Lat_baze,distance_m
4253,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135
4695,Escola Básica do 1º Ciclo e Jardim de Infância...,41.204380,-8.568072,EB Pícua,-8.568378,41.204116,38.947084
4741,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859
4773,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765
4821,Escola Básica e Secundária do Levante da Maia,41.229515,-8.594331,EBS Levante,-8.593993,41.229248,40.970777
4869,Escola Básica e Secundária de Pedrouços,41.191517,-8.585074,EBS Pedrouços,-8.585438,41.190956,69.385754
5025,Centro Escolar da Gandra,41.216359,-8.561883,EB Gandra,-8.562116,41.216358,19.422158
5140,Escola Básica da Maia,41.226554,-8.619460,EB Maia,-8.619240,41.226872,39.970016
5241,Escola Básica Gonçalo Mendes da Maia,41.232667,-8.614885,EB23 Gonçalo Mendes da Maia,-8.615349,41.232819,42.292540
5297,Escola Básica n.º 1 de Gueifães,41.226159,-8.607879,EB Gueifães 1 - Centro Escolar,-8.608060,41.226196,15.673916


In [ ]:
baze_escolas_OSM['key'] = 1
contadores_2_coord['key'] = 1
cross = pd.merge(baze_escolas_OSM, contadores_2_coord, on='key').drop('key', axis=1)

cross['distance_m_2'] = cross.apply(
    lambda row: min(haversine(row['Lat_escolas_OSM'], row['Lon_escolas_OSM'], row['Lon'], row['Lat']), haversine(row['Lat_baze'], row['Lon_baze'], row['Lon'], row['Lat'])),
    axis=1
)

baze_escolas_OSM_contadores_2 = cross[cross['distance_m_2'] <= distance_threshold]

baze_escolas_OSM_contadores_2 = baze_escolas_OSM_contadores_2.rename(columns={
    'Lon':'Latitude_contadores_2',
    'Lat':'Longitude_contadores_2',
    'desig_edificio':'designação_contadores_2',
    'Nome da Escola':'Escola_contadores_2'
})

baze_escolas_OSM_contadores_2

/tmp/ipython-input-41-2919081624.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contadores_2_coord['key'] = 1


,Name_escolas_OSM,Lat_escolas_OSM,Lon_escolas_OSM,designação,Lon_baze,Lat_baze,distance_m,Longitude_contadores_2,Latitude_contadores_2,designação_contadores_2,Escola_contadores_2,distance_m_2
22,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614856,41.229920,NaN,NaN,32.028804
280,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614838,41.230097,Rega,NaN,51.465991
593,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614863,41.230080,Rega-Jardim Capela,NaN,49.750957
768,Escola Básica do 1º Ciclo e Jardim de Infância...,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614928,41.230040,Escola EB1/JI Cidade Jardim,EB de Cidade Jardim,46.290311
2617,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859,-8.570950,41.211343,Rega,EB de Moutidos,17.521407
2618,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859,-8.570950,41.211343,NaN,EB de Moutidos,17.521407
2619,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859,-8.570950,41.211343,NaN,EB de Moutidos,17.521407
2813,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765,-8.574324,41.204180,Pavilhao Municipal Aguas Santa,NaN,24.758865
3054,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765,-8.574356,41.204107,Urb Qta Caverneira-Rega,NaN,31.165156
3536,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765,-8.574357,41.204140,Escola Secundária (Incendio),EB e Secundária de Águas Santas,29.206907


### Análise

In [ ]:
print("Nº de entradas de Contadores_2:", len(contadores_2_coord))
print("Nº de entradas de baze:", len(baze))
print("Nº de entradas de escolas_OSM:", len(escolas_OSM))

Nº de entradas de Contadores_2: 896
Nº de entradas de baze: 47
Nº de entradas de escolas_OSM: 377


Nesta experiência foram feitas 3 versões do matching:
- **Caso A**: (contadores_2 ⨝ baze) ⨝ escolas_OSM
- **Caso B**: (contadores_2 ⨝ escolas_OSM) ⨝ baze
- **Caso C**: (baze ⨝ escolas_OSM) ⨝ contadores_2

#### Normalização dos dados

In [ ]:
replacements = {
    'escola ': 'e',
    ' escolar': 'e',
    'esc ': 'e',
    'secundaria': 's',
    'basica': 'b',
    'sec ': 's',
    'centro ': 'c',
    'jardim': 'j',
    'infancia':'i',
    ' de ': ' ',
    ' do ':' ',
    ' e ': ' ',
    ' com ': ' ',
    'n.o ': '',
    'ciclo ': '',
    '1.o': '1',
    '1o': '1',
}

def normalize(text):
    if pd.isna(text):
        return ""
    text = ''.join(c for c in unicodedata.normalize('NFKD', str(text)) if not unicodedata.combining(c))

    text = text.lower()

    for old, new in replacements.items():
        text = text.replace(old, new)

    return ' '.join(text.split())

threshold_c_b_e = 60


#### Caso A

In [ ]:
print("Nº de entradas da tabela intermédia:", len(contadores_baze_2))
print("Nº de entradas da tabela resultado:", len(contadores_baze_escolas_OSM_2))

Nº de entradas da tabela intermédia: 51
Nº de entradas da tabela resultado: 49


In [ ]:
results = []

for i in range(len(contadores_baze_escolas_OSM_2)):
    row = contadores_baze_escolas_OSM_2.iloc[i]

    designação_contadores_2 = normalize(row['designação_contadores_2'])
    Escola_contadores_2 = normalize(row['Escola_contadores_2'])
    designação_baze = normalize(row['designação'])
    Name_escolas_OSM = normalize(row['Name_escolas_OSM'])

    if pd.isna(Name_escolas_OSM) or (pd.isna(designação_contadores_2) and pd.isna(Escola_contadores_2) and pd.isna(designação_baze)):
        results.append('non_identified')
        continue

    scores = []
    scores.append(fuzz.ratio(designação_contadores_2, Name_escolas_OSM))
    scores.append(fuzz.ratio(Escola_contadores_2, Name_escolas_OSM))
    scores.append(fuzz.ratio(designação_baze, Name_escolas_OSM))

    if max(scores) >= threshold_c_b_e:
        results.append('true_positive')
    else:
        results.append('false_positive')

contadores_baze_escolas_OSM_2['result'] = results

result_counts = contadores_baze_escolas_OSM_2['result'].value_counts()

print("\nCounts:")
print(result_counts)

display(HTML(contadores_baze_escolas_OSM_2.to_html()))


Counts:
result
true_positive     48
false_positive     1
Name: count, dtype: int64


,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,designação,Lon_baze,Lat_baze,distance_m,Name_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM,distance_m_2,result
129,-8.658449,41.234673,Escola EB1/JI Crestins,EB de Crestins,Eb Crestins,-8.658670,41.234562,22.170356,Creche/Pré-Escolar de Crestins,41.234286,-8.659379,66.837473,false_positive
271,-8.658449,41.234673,Escola EB1/JI Crestins,EB de Crestins,Eb Crestins,-8.658670,41.234562,22.170356,Escola Básica do 1º Ciclo de Crestins,41.234412,-8.658729,17.463567,true_positive
530,-8.614466,41.259500,Escola EB1/JI Castelo Maia,"EB do Castelo da Maia, Santa Maria De Avioso",EB Castelo da Maia,-8.614067,41.259409,34.847964,Escola Básica do 1º Ciclo e Jardim de Infância do Castêlo da Maia,41.259417,-8.613878,15.868954,true_positive
844,-8.614856,41.229920,NaN,NaN,EB Cidade Jardim,-8.614799,41.229635,32.028804,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,21.963135,true_positive
1276,-8.590447,41.183175,Fonte Luminosa Rotunda G M,NaN,EB Pedrouços,-8.589668,41.183069,66.271312,Escola Básica n.º 2 de Pedrouços,41.183170,-8.589583,13.308781,true_positive
1609,-8.574324,41.204180,Pavilhao Municipal Aguas Santa,NaN,EBS Águas Santas,-8.574054,41.204271,24.758865,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,47.022765,true_positive
2164,-8.663104,41.243864,NaN,NaN,EB1 Pedras Rubras,-8.663562,41.244178,51.796339,Escola Básica do 1º Ciclo e Jardim de Infância de Pedras Rubras,41.244295,-8.663441,16.539937,true_positive
2408,-8.592291,41.184313,Totalizador,NaN,EB Enxurreiras,-8.592795,41.184203,43.934742,Escola Básica do 1º Ciclo com Jardim de Infância de Enxurreiras,41.184123,-8.593053,23.344453,true_positive
2785,-8.592291,41.184313,Serviços Comuns,NaN,EB Enxurreiras,-8.592795,41.184203,43.934742,Escola Básica do 1º Ciclo com Jardim de Infância de Enxurreiras,41.184123,-8.593053,23.344453,true_positive
3106,-8.614838,41.230097,Rega,NaN,EB Cidade Jardim,-8.614799,41.229635,51.465991,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,21.963135,true_positive


#### Caso B

In [ ]:
print("Nº de entradas da tabela intermédia:", len(contadores_escolas_OSM_2))
print("Nº de entradas da tabela resultado:", len(contadores_escola_OSM_baze_2))

Nº de entradas da tabela intermédia: 54
Nº de entradas da tabela resultado: 36


In [ ]:
results = []

for i in range(len(contadores_escola_OSM_baze_2)):
    row = contadores_escola_OSM_baze_2.iloc[i]

    designação_contadores_2 = normalize(row['designação_contadores_2'])
    Escola_contadores_2 = normalize(row['Escola_contadores_2'])
    designação_baze = normalize(row['designação_baze'])
    Name_escolas_OSM = normalize(row['Name_escolas_OSM'])

    if pd.isna(designação_baze) or (pd.isna(designação_contadores_2) and pd.isna(Escola_contadores_2) and pd.isna(Name_escolas_OSM)):
        results.append('non_identified')
        continue

    scores = []
    scores.append(fuzz.ratio(designação_contadores_2, designação_baze))
    scores.append(fuzz.ratio(Escola_contadores_2, designação_baze))
    scores.append(fuzz.ratio(Name_escolas_OSM, designação_baze))

    if max(scores) >= threshold_c_b_e:
        results.append('true_positive')
    else:
        results.append('false_positive')

contadores_escola_OSM_baze_2['result'] = results

result_counts = contadores_escola_OSM_baze_2['result'].value_counts()

print("\nCounts:")
print(result_counts)

display(HTML(contadores_escola_OSM_baze_2.to_html()))


Counts:
result
true_positive    36
Name: count, dtype: int64


,Name_escolas_OSM,Lat_escolas_OSM,Lon_escolas_OSM,Lon_contadores_2,Lat_contadores_2,designação_contadores_2,Escola_contadores_2,distance_m,designação_baze,Longitude_baze,Latitude_baze,distance_m_2,result
446,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,-8.614856,41.229920,NaN,NaN,48.147922,EB Cidade Jardim,-8.614799,41.229635,21.963135,true_positive
493,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,-8.614838,41.230097,Rega,NaN,67.728254,EB Cidade Jardim,-8.614799,41.229635,21.963135,true_positive
540,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,-8.614863,41.230080,Rega-Jardim Capela,NaN,65.493747,EB Cidade Jardim,-8.614799,41.229635,21.963135,true_positive
587,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,-8.614928,41.230040,Escola EB1/JI Cidade Jardim,EB de Cidade Jardim,60.413161,EB Cidade Jardim,-8.614799,41.229635,21.963135,true_positive
652,Escola Básica de Moutidos,41.211528,-8.570596,-8.570950,41.211343,Rega,EB de Moutidos,36.093509,EB Moutidos,-8.570920,41.211187,17.521407,true_positive
699,Escola Básica de Moutidos,41.211528,-8.570596,-8.570950,41.211343,NaN,EB de Moutidos,36.093509,EB Moutidos,-8.570920,41.211187,17.521407,true_positive
746,Escola Básica de Moutidos,41.211528,-8.570596,-8.570950,41.211343,NaN,EB de Moutidos,36.093509,EB Moutidos,-8.570920,41.211187,17.521407,true_positive
795,Centro Escolar da Gandra,41.216359,-8.561883,-8.562471,41.216511,Escola Gandra,EB de Gandra,51.966741,EB Gandra,-8.562116,41.216358,19.422158,true_positive
842,Centro Escolar da Gandra,41.216359,-8.561883,-8.562471,41.216511,Escola Gandra,EB de Gandra,51.966741,EB Gandra,-8.562116,41.216358,19.422158,true_positive
889,Centro Escolar da Gandra,41.216359,-8.561883,-8.562471,41.216511,Escola Gandra,EB de Gandra,51.966741,EB Gandra,-8.562116,41.216358,19.422158,true_positive


#### Caso C

In [ ]:
print("Nº de entradas da tabela intermédia:", len(baze_escolas_OSM))
print("Nº de entradas da tabela resultado:", len(baze_escolas_OSM_contadores_2))

Nº de entradas da tabela intermédia: 45
Nº de entradas da tabela resultado: 51


In [ ]:
results = []

for i in range(len(baze_escolas_OSM_contadores_2)):
    row = baze_escolas_OSM_contadores_2.iloc[i]

    designação_contadores_2 = normalize(row['designação_contadores_2'])
    Escola_contadores_2 = normalize(row['Escola_contadores_2'])
    designação_baze = normalize(row['designação'])
    Name_escolas_OSM = normalize(row['Name_escolas_OSM'])

    if (pd.isna(designação_contadores_2) and pd.isna(Escola_contadores_2)) or (pd.isna(designação_baze) and pd.isna(Name_escolas_OSM)):
        results.append('non_identified')
        continue

    scores = []
    scores.append(fuzz.ratio(designação_baze, designação_contadores_2))
    scores.append(fuzz.ratio(Name_escolas_OSM, designação_contadores_2))
    scores.append(fuzz.ratio(designação_baze, Escola_contadores_2))
    scores.append(fuzz.ratio(Name_escolas_OSM, Escola_contadores_2))

    if max(scores) >= threshold_c_b_e:
        results.append('true_positive')
    else:
        results.append('false_positive')

baze_escolas_OSM_contadores_2['result'] = results

result_counts = baze_escolas_OSM_contadores_2['result'].value_counts()

print("\nCounts:")
print(result_counts)

display(HTML(baze_escolas_OSM_contadores_2.to_html()))


Counts:
result
true_positive     30
false_positive    21
Name: count, dtype: int64


,Name_escolas_OSM,Lat_escolas_OSM,Lon_escolas_OSM,designação,Lon_baze,Lat_baze,distance_m,Longitude_contadores_2,Latitude_contadores_2,designação_contadores_2,Escola_contadores_2,distance_m_2,result
22,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614856,41.229920,NaN,NaN,32.028804,false_positive
280,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614838,41.230097,Rega,NaN,51.465991,false_positive
593,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614863,41.230080,Rega-Jardim Capela,NaN,49.750957,false_positive
768,Escola Básica do 1º Ciclo e Jardim de Infância de Cidade Jardim,41.229499,-8.614988,EB Cidade Jardim,-8.614799,41.229635,21.963135,-8.614928,41.230040,Escola EB1/JI Cidade Jardim,EB de Cidade Jardim,46.290311,true_positive
2617,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859,-8.570950,41.211343,Rega,EB de Moutidos,17.521407,true_positive
2618,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859,-8.570950,41.211343,NaN,EB de Moutidos,17.521407,true_positive
2619,Escola Básica de Moutidos,41.211528,-8.570596,EB Moutidos,-8.570920,41.211187,46.645859,-8.570950,41.211343,NaN,EB de Moutidos,17.521407,true_positive
2813,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765,-8.574324,41.204180,Pavilhao Municipal Aguas Santa,NaN,24.758865,false_positive
3054,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765,-8.574356,41.204107,Urb Qta Caverneira-Rega,NaN,31.165156,false_positive
3536,Escola Básica e Secundária de Águas Santas,41.204334,-8.573498,EBS Águas Santas,-8.574054,41.204271,47.022765,-8.574357,41.204140,Escola Secundária (Incendio),EB e Secundária de Águas Santas,29.206907,true_positive


#### Conclusão

O facto de a ordem do join mudar o número de entradas nas tabelas intermédias e resultado demonstra que não existe associatividade nesta operação.
Isto deve-se ao critério usado ser um threshold para a proximidade de coordenadas.

## Tabela resultado final

### Construção da tabela

Primeira coluna ***Nome do edificio*** feita com todos os valores não nulos das tabelas utilizadas: ***edificios_imoveis***, ***filtered_edificios***, ***filtered_contadores_2***, ***baze***, ***escolas_OSM***. Adição de uma coluna Flag para ajudar a distinguir a origem dos valores em relação às tabelas.

In [ ]:
valid_contadores_2 = filtered_contadores_2.dropna(subset=['Nome da Escola'])

valid_edificios = filtered_edificios.dropna(subset=['Designação'])

tabela_resultado = pd.concat([
    pd.Series(edificios_imoveis['designation_imoveis'], name='Nome do edificio').to_frame().assign(Flag='I'),
    pd.Series(valid_edificios['Designação'], name='Nome do edificio').to_frame().assign(Flag='E'),
    pd.Series(valid_contadores_2['Nome da Escola'], name='Nome do edificio').to_frame().assign(Flag='C'),
    pd.Series(baze['designação'], name='Nome do edificio').to_frame().assign(Flag='B'),
    pd.Series(escolas_OSM['Name'], name='Nome do edificio').to_frame().assign(Flag='OSM')
], ignore_index=True)

Preparação de mais algumas tabelas com a remoção dos valores nulos

In [ ]:
contadores_baze_2 = contadores_baze_2.dropna(subset=['Escola_contadores_2'])

contadores_escolas_OSM_2 = contadores_escolas_OSM_2.dropna(subset=['Escola_contadores_2'])

imo_edi_con_2 = imo_edi_con_2.dropna(subset=['Escola_contadores_2'])

Remoção de duplicados nas tabelas de matching que vão preencher a tabela resultado

In [ ]:
edificios_imoveis = edificios_imoveis.drop_duplicates(subset='designation_imoveis', keep='first')

edificios_imoveis = edificios_imoveis.drop_duplicates(subset='designation_edificios', keep='first')

edificios_contadores_2 = edificios_contadores_2.drop_duplicates(subset='designation_edificios', keep='first')

edificios_contadores_2 = edificios_contadores_2.drop_duplicates(subset='designation_contadores_2', keep='first')

edificios_baze = edificios_baze.drop_duplicates(subset='designation_edificios', keep='first')

edificios_baze = edificios_baze.drop_duplicates(subset='designation_baze', keep='first')

edificios_escolas_OSM = edificios_escolas_OSM.drop_duplicates(subset='designation_edificios', keep='first')

edificios_escolas_OSM = edificios_escolas_OSM.drop_duplicates(subset='designation_escolas_OSM', keep='first')

contadores_baze_2 = contadores_baze_2.drop_duplicates(subset='Escola_contadores_2', keep='first')

contadores_baze_2 = contadores_baze_2.drop_duplicates(subset='designação', keep='first')

contadores_escolas_OSM_2 = contadores_escolas_OSM_2.drop_duplicates(subset='Name_escolas_OSM', keep='first')

contadores_escolas_OSM_2 = contadores_escolas_OSM_2.drop_duplicates(subset='Escola_contadores_2', keep='first')

baze_escolas_OSM = baze_escolas_OSM.drop_duplicates(subset='Name_escolas_OSM', keep='first')

baze_escolas_OSM = baze_escolas_OSM.drop_duplicates(subset='designação', keep='first')

imo_edi_con_2 = imo_edi_con_2.drop_duplicates(subset='designation_imoveis', keep='first')

imo_edi_con_2 = imo_edi_con_2.drop_duplicates(subset='Escola_contadores_2', keep='first')

imo_edi_baze = imo_edi_baze.drop_duplicates(subset='designation_imoveis', keep='first')

imo_edi_baze = imo_edi_baze.drop_duplicates(subset='designação', keep='first')

imo_edi_escolas_OSM = imo_edi_escolas_OSM.drop_duplicates(subset='designation_imoveis', keep='first')

imo_edi_escolas_OSM = imo_edi_escolas_OSM.drop_duplicates(subset='Name', keep='first')

valid_contadores_2 = valid_contadores_2.drop_duplicates(subset='Nome da Escola', keep='first')

escolas_OSM = escolas_OSM.drop_duplicates(subset='Name', keep='first')

Adição à tabela resultado as colunas que ajudam à comparação de todas as tabelas que foram utilizadas.

In [ ]:
tabela_resultado[['Nome_imoveis','Nº_inventario_imoveis', 'Nome_edificios', 'Nº_inventario_edificios', 'Latitude_edificios', 'Longitude_edificios', 'Nome_contadores_2', 'Latitude_contadores_2', 'Longitude_contadores_2', 'Nome_baze', 'Latitude_baze', 'Longitude_baze', 'Nome_escolas_OSM', 'Latitude_escolas_OSM', 'Longitude_escolas_OSM']]=''

tabela_resultado.head()

,Nome do edificio,Flag,Nome_imoveis,Nº_inventario_imoveis,Nome_edificios,Nº_inventario_edificios,Latitude_edificios,Longitude_edificios,Nome_contadores_2,Latitude_contadores_2,Longitude_contadores_2,Nome_baze,Latitude_baze,Longitude_baze,Nome_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM
0,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CAS...,I,,,,,,,,,,,,,,,
1,EDIFICIO DO FORUM DA MAIA,I,,,,,,,,,,,,,,,
2,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,I,,,,,,,,,,,,,,,
3,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS D...,I,,,,,,,,,,,,,,,
4,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS...,I,,,,,,,,,,,,,,,


### Inserção dos valores na tabela resultado através de todo o matching feito

#### Entradas relativas às tabelas ***edificio_imoveis*** e ***filtered_edificios***

In [ ]:
merged = tabela_resultado.merge(edificios_imoveis, left_on='Nome do edificio', right_on='designation_imoveis', how='left')


tabela_resultado['Nome_imoveis']=merged['designation_imoveis'].combine_first(tabela_resultado['Nome_imoveis'])
tabela_resultado['Nº_inventario_imoveis']= merged['number_imoveis'].combine_first(tabela_resultado['Nº_inventario_imoveis'])

tabela_resultado['Nome_edificios']=merged['designation_edificios'].combine_first(tabela_resultado['Nome_edificios'])
tabela_resultado['Nº_inventario_edificios']= merged['number_edificios'].combine_first(tabela_resultado['Nº_inventario_edificios'])
tabela_resultado['Latitude_edificios']= merged['Lat_edificios'].combine_first(tabela_resultado['Latitude_edificios'])
tabela_resultado['Longitude_edificios']= merged['Lon_edificios'].combine_first(tabela_resultado['Longitude_edificios'])

In [ ]:
merged = tabela_resultado.merge(edificios_imoveis, left_on='Nome do edificio', right_on='designation_edificios', how='left')

tabela_resultado['Nome_imoveis']=merged['designation_imoveis'].combine_first(tabela_resultado['Nome_imoveis'])
tabela_resultado['Nº_inventario_imoveis']= merged['number_imoveis'].combine_first(tabela_resultado['Nº_inventario_imoveis'])


#### Entradas relativas às tabelas ***filtered_contadores_2*** e ***filtered_edificios***

In [ ]:
merged = tabela_resultado.merge(edificios_contadores_2, left_on='Nome do edificio', right_on='designation_edificios', how='left')

tabela_resultado['Nome_contadores_2']=merged['designation_contadores_2'].combine_first(tabela_resultado['Nome_contadores_2'])
tabela_resultado['Latitude_contadores_2']=merged['lat_contadores_2'].combine_first(tabela_resultado['Latitude_contadores_2'])
tabela_resultado['Longitude_contadores_2']=merged['lon_contadores_2'].combine_first(tabela_resultado['Longitude_contadores_2'])


In [ ]:
merged = tabela_resultado.merge(edificios_contadores_2, left_on='Nome do edificio', right_on='designation_contadores_2', how='left')

tabela_resultado['Nome_edificios']=merged['designation_edificios'].combine_first(tabela_resultado['Nome_edificios'])
tabela_resultado['Latitude_edificios']=merged['lat_edificios'].combine_first(tabela_resultado['Latitude_edificios'])
tabela_resultado['Longitude_edificios']=merged['lon_edificios'].combine_first(tabela_resultado['Longitude_edificios'])

#### Entradas relativas às tabelas ***baze*** e ***filtered_edificios***

In [ ]:
merged = tabela_resultado.merge(edificios_baze, left_on='Nome do edificio', right_on='designation_edificios', how='left')

tabela_resultado['Nome_baze']=merged['designation_baze'].combine_first(tabela_resultado['Nome_baze'])
tabela_resultado['Latitude_baze']=merged['lat_baze'].combine_first(tabela_resultado['Latitude_baze'])
tabela_resultado['Longitude_baze']=merged['lon_baze'].combine_first(tabela_resultado['Longitude_baze'])

In [ ]:
merged = tabela_resultado.merge(edificios_baze, left_on='Nome do edificio', right_on='designation_baze', how='left')

tabela_resultado['Nome_edificios']=merged['designation_edificios'].combine_first(tabela_resultado['Nome_edificios'])
tabela_resultado['Latitude_edificios']=merged['lat_edificios'].combine_first(tabela_resultado['Latitude_edificios'])
tabela_resultado['Longitude_edificios']=merged['lon_edificios'].combine_first(tabela_resultado['Longitude_edificios'])

#### Entradas relativas às tabelas ***escolas_OSM*** e ***filtered_edificios***

In [ ]:
merged = tabela_resultado.merge(edificios_escolas_OSM, left_on='Nome do edificio', right_on='designation_edificios', how='left')

tabela_resultado['Nome_escolas_OSM']=merged['designation_escolas_OSM'].combine_first(tabela_resultado['Nome_escolas_OSM'])
tabela_resultado['Latitude_escolas_OSM']=merged['lat_escolas_OSM'].combine_first(tabela_resultado['Latitude_escolas_OSM'])
tabela_resultado['Longitude_escolas_OSM']=merged['lon_escolas_OSM'].combine_first(tabela_resultado['Longitude_escolas_OSM'])

In [ ]:
merged = tabela_resultado.merge(edificios_escolas_OSM, left_on='Nome do edificio', right_on='designation_escolas_OSM', how='left')

tabela_resultado['Nome_edificios']=merged['designation_edificios'].combine_first(tabela_resultado['Nome_edificios'])
tabela_resultado['Latitude_edificios']=merged['lat_edificios'].combine_first(tabela_resultado['Latitude_edificios'])
tabela_resultado['Longitude_edificios']=merged['lon_edificios'].combine_first(tabela_resultado['Longitude_edificios'])

#### Entradas relativas às tabelas ***baze*** e ***filtered_contadores_2***

In [ ]:
merged = tabela_resultado.merge(contadores_baze_2, left_on='Nome do edificio', right_on='Escola_contadores_2', how='left')

tabela_resultado['Nome_baze']=merged['designação'].combine_first(tabela_resultado['Nome_baze'])
tabela_resultado['Latitude_baze']=merged['Lat_baze'].combine_first(tabela_resultado['Latitude_baze'])
tabela_resultado['Longitude_baze']=merged['Lon_baze'].combine_first(tabela_resultado['Longitude_baze'])

In [ ]:
merged = tabela_resultado.merge(contadores_baze_2, left_on='Nome do edificio', right_on='designação', how='left')

tabela_resultado['Nome_contadores_2']=merged['Escola_contadores_2'].combine_first(tabela_resultado['Nome_contadores_2'])
tabela_resultado['Latitude_contadores_2']=merged['Lat_contadores_2'].combine_first(tabela_resultado['Latitude_contadores_2'])
tabela_resultado['Longitude_contadores_2']=merged['Lon_contadores_2'].combine_first(tabela_resultado['Longitude_contadores_2'])

#### Entradas relativas às tabelas ***escolas_OSM*** e ***filtered_contadores_2***

In [ ]:
merged = tabela_resultado.merge(contadores_escolas_OSM_2, left_on='Nome do edificio', right_on='Escola_contadores_2', how='left')

tabela_resultado['Nome_escolas_OSM']=merged['Name_escolas_OSM'].combine_first(tabela_resultado['Nome_escolas_OSM'])
tabela_resultado['Latitude_escolas_OSM']=merged['Lat_escolas_OSM'].combine_first(tabela_resultado['Latitude_escolas_OSM'])
tabela_resultado['Longitude_escolas_OSM']=merged['Lon_escolas_OSM'].combine_first(tabela_resultado['Longitude_escolas_OSM'])

In [ ]:
merged = tabela_resultado.merge(contadores_escolas_OSM_2, left_on='Nome do edificio', right_on='Name_escolas_OSM', how='left')

tabela_resultado['Nome_contadores_2']=merged['Escola_contadores_2'].combine_first(tabela_resultado['Nome_contadores_2'])
tabela_resultado['Latitude_contadores_2']=merged['Lat_contadores_2'].combine_first(tabela_resultado['Latitude_contadores_2'])
tabela_resultado['Longitude_contadores_2']=merged['Lon_contadores_2'].combine_first(tabela_resultado['Longitude_contadores_2'])

#### Entradas relativas às tabelas ***baze*** e ***escolas_OSM***

In [ ]:
merged = tabela_resultado.merge(baze_escolas_OSM, left_on='Nome do edificio', right_on='designação', how='left')

tabela_resultado['Nome_escolas_OSM']=merged['Name_escolas_OSM'].combine_first(tabela_resultado['Nome_escolas_OSM'])
tabela_resultado['Latitude_escolas_OSM']=merged['Lat_escolas_OSM'].combine_first(tabela_resultado['Latitude_escolas_OSM'])
tabela_resultado['Longitude_escolas_OSM']=merged['Lon_escolas_OSM'].combine_first(tabela_resultado['Longitude_escolas_OSM'])

In [ ]:
merged = tabela_resultado.merge(baze_escolas_OSM, left_on='Nome do edificio', right_on='Name_escolas_OSM', how='left')

tabela_resultado['Nome_baze']=merged['designação'].combine_first(tabela_resultado['Nome_baze'])
tabela_resultado['Latitude_baze']=merged['Lat_baze'].combine_first(tabela_resultado['Latitude_baze'])
tabela_resultado['Longitude_baze']=merged['Lon_baze'].combine_first(tabela_resultado['Longitude_baze'])

#### Entradas relativas às tabelas ***edificios_imoveis*** e ***filtered_contadores_2***

In [ ]:
merged = tabela_resultado.merge(imo_edi_con_2, left_on='Nome do edificio', right_on='designation_imoveis', how='left')

tabela_resultado['Nome_contadores_2']=merged['Escola_contadores_2'].combine_first(tabela_resultado['Nome_contadores_2'])
tabela_resultado['Latitude_contadores_2']=merged['Lat_contadores_2'].combine_first(tabela_resultado['Latitude_contadores_2'])
tabela_resultado['Longitude_contadores_2']=merged['Lon_contadores_2'].combine_first(tabela_resultado['Longitude_contadores_2'])

In [ ]:
merged = tabela_resultado.merge(imo_edi_con_2, left_on='Nome do edificio', right_on='Escola_contadores_2', how='left')

tabela_resultado['Nome_imoveis']=merged['designation_imoveis'].combine_first(tabela_resultado['Nome_imoveis'])
tabela_resultado['Nº_inventario_imoveis']=merged['Nº inventario'].combine_first(tabela_resultado['Nº_inventario_imoveis'])

#### Entradas relativas às tabelas ***edificios_imoveis*** e ***baze***

In [ ]:
merged = tabela_resultado.merge(imo_edi_baze, left_on='Nome do edificio', right_on='designation_imoveis', how='left')

tabela_resultado['Nome_baze']=merged['designação'].combine_first(tabela_resultado['Nome_baze'])
tabela_resultado['Latitude_baze']=merged['lat_baze'].combine_first(tabela_resultado['Latitude_baze'])
tabela_resultado['Longitude_baze']=merged['lon_baze'].combine_first(tabela_resultado['Longitude_baze'])

In [ ]:
merged = tabela_resultado.merge(imo_edi_baze, left_on='Nome do edificio', right_on='designação', how='left')

tabela_resultado['Nome_imoveis']=merged['designation_imoveis'].combine_first(tabela_resultado['Nome_imoveis'])
tabela_resultado['Nº_inventario_imoveis']=merged['Nº inventario'].combine_first(tabela_resultado['Nº_inventario_imoveis'])

#### Entradas relativas às tabelas ***edificios_imoveis*** e ***escolas_OSM***

In [ ]:
merged = tabela_resultado.merge(imo_edi_escolas_OSM, left_on='Nome do edificio', right_on='designation_imoveis', how='left')

tabela_resultado['Nome_escolas_OSM']=merged['Name'].combine_first(tabela_resultado['Nome_escolas_OSM'])
tabela_resultado['Latitude_escolas_OSM']=merged['lat_escolas_OSM'].combine_first(tabela_resultado['Latitude_escolas_OSM'])
tabela_resultado['Longitude_escolas_OSM']=merged['lon_escolas_OSM'].combine_first(tabela_resultado['Longitude_escolas_OSM'])

In [ ]:
merged = tabela_resultado.merge(imo_edi_escolas_OSM, left_on='Nome do edificio', right_on='Name', how='left')

tabela_resultado['Nome_imoveis']=merged['designation_imoveis'].combine_first(tabela_resultado['Nome_imoveis'])
tabela_resultado['Nº_inventario_imoveis']=merged['Nº inventario'].combine_first(tabela_resultado['Nº_inventario_imoveis'])

#### Entradas relativas às mesmas tabelas

In [ ]:
merged = tabela_resultado.merge(valid_edificios, left_on='Nome do edificio', right_on='Designação', how='left')


tabela_resultado['Nome_edificios']=merged['Designação'].combine_first(tabela_resultado['Nome_edificios'])
tabela_resultado['Nº_inventario_edificios']= merged['N.º Inventário'].combine_first(tabela_resultado['Nº_inventario_edificios'])
tabela_resultado['Latitude_edificios']= merged['Latitude'].combine_first(tabela_resultado['Latitude_edificios'])
tabela_resultado['Longitude_edificios']= merged['Longitude'].combine_first(tabela_resultado['Longitude_edificios'])

display(HTML(tabela_resultado.to_html()))

,Nome do edificio,Flag,Nome_imoveis,Nº_inventario_imoveis,Nome_edificios,Nº_inventario_edificios,Latitude_edificios,Longitude_edificios,Nome_contadores_2,Latitude_contadores_2,Longitude_contadores_2,Nome_baze,Latitude_baze,Longitude_baze,Nome_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM
0,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),I,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),5814,Mercado do Castêlo,5814,,,,,,,,,,,
1,EDIFICIO DO FORUM DA MAIA,I,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,6972,41.234,-8.622233,,,,,,,,,
2,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,I,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,6973,,,,,,,,,,,
3,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,I,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,6975,Paços do Concelho da Maia,6975,41.233341,-8.622518,,,,,,,,,
4,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),I,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),24755,Torre Lidador - Câmara Municipal da Maia,24755,41.233396,-8.622039,,,,,,,,,
5,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),I,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),38754,Complexo Municipal de Piscinas de Gueifães,38754,41.214994,-8.600761,EB de Gueifães,41.214185,-8.600243,,,,,,
6,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",I,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",45234,Creche/ Pré-Escolar de Águas Santas 2,45234,,,,,,,,,,,
7,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",I,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",46678,Polidesportivo Municipal do Lidador,46678,,,,,,,,,,,
8,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),I,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),46679,Polidesportivo Monte Faro,46679,,,,,,,,,,,
9,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",I,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",46680,Polidesportivo Municipal dos Moutidos,46680,,,,,,,,,,,


In [ ]:
merged = tabela_resultado.merge(valid_contadores_2, left_on='Nome do edificio', right_on='Nome da Escola', how='left')


tabela_resultado['Nome_contadores_2']=merged['Nome da Escola'].combine_first(tabela_resultado['Nome_contadores_2'])
tabela_resultado['Longitude_contadores_2']= merged['Lat'].combine_first(tabela_resultado['Longitude_contadores_2'])
tabela_resultado['Latitude_contadores_2']= merged['Lon'].combine_first(tabela_resultado['Latitude_contadores_2'])

display(HTML(tabela_resultado.to_html()))

,Nome do edificio,Flag,Nome_imoveis,Nº_inventario_imoveis,Nome_edificios,Nº_inventario_edificios,Latitude_edificios,Longitude_edificios,Nome_contadores_2,Latitude_contadores_2,Longitude_contadores_2,Nome_baze,Latitude_baze,Longitude_baze,Nome_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM
0,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),I,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),5814,Mercado do Castêlo,5814,,,,,,,,,,,
1,EDIFICIO DO FORUM DA MAIA,I,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,6972,41.234,-8.622233,,,,,,,,,
2,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,I,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,6973,,,,,,,,,,,
3,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,I,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,6975,Paços do Concelho da Maia,6975,41.233341,-8.622518,,,,,,,,,
4,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),I,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),24755,Torre Lidador - Câmara Municipal da Maia,24755,41.233396,-8.622039,,,,,,,,,
5,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),I,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),38754,Complexo Municipal de Piscinas de Gueifães,38754,41.214994,-8.600761,EB de Gueifães,41.214185,-8.600243,,,,,,
6,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",I,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",45234,Creche/ Pré-Escolar de Águas Santas 2,45234,,,,,,,,,,,
7,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",I,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",46678,Polidesportivo Municipal do Lidador,46678,,,,,,,,,,,
8,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),I,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),46679,Polidesportivo Monte Faro,46679,,,,,,,,,,,
9,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",I,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",46680,Polidesportivo Municipal dos Moutidos,46680,,,,,,,,,,,


In [ ]:
merged = tabela_resultado.merge(baze, left_on='Nome do edificio', right_on='designação', how='left')


tabela_resultado['Nome_baze']=merged['designação'].combine_first(tabela_resultado['Nome_baze'])
tabela_resultado['Latitude_baze']= merged['latitude'].combine_first(tabela_resultado['Latitude_baze'])
tabela_resultado['Longitude_baze']= merged['longitude'].combine_first(tabela_resultado['Longitude_baze'])

display(HTML(tabela_resultado.to_html()))

,Nome do edificio,Flag,Nome_imoveis,Nº_inventario_imoveis,Nome_edificios,Nº_inventario_edificios,Latitude_edificios,Longitude_edificios,Nome_contadores_2,Latitude_contadores_2,Longitude_contadores_2,Nome_baze,Latitude_baze,Longitude_baze,Nome_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM
0,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),I,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),5814,Mercado do Castêlo,5814,,,,,,,,,,,
1,EDIFICIO DO FORUM DA MAIA,I,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,6972,41.234,-8.622233,,,,,,,,,
2,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,I,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,6973,,,,,,,,,,,
3,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,I,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,6975,Paços do Concelho da Maia,6975,41.233341,-8.622518,,,,,,,,,
4,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),I,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),24755,Torre Lidador - Câmara Municipal da Maia,24755,41.233396,-8.622039,,,,,,,,,
5,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),I,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),38754,Complexo Municipal de Piscinas de Gueifães,38754,41.214994,-8.600761,EB de Gueifães,41.214185,-8.600243,,,,,,
6,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",I,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",45234,Creche/ Pré-Escolar de Águas Santas 2,45234,,,,,,,,,,,
7,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",I,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",46678,Polidesportivo Municipal do Lidador,46678,,,,,,,,,,,
8,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),I,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),46679,Polidesportivo Monte Faro,46679,,,,,,,,,,,
9,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",I,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",46680,Polidesportivo Municipal dos Moutidos,46680,,,,,,,,,,,


In [ ]:
merged = tabela_resultado.merge(escolas_OSM, left_on='Nome do edificio', right_on='Name', how='left')


tabela_resultado['Nome_escolas_OSM']=merged['Name'].combine_first(tabela_resultado['Nome_escolas_OSM'])
tabela_resultado['Latitude_escolas_OSM']= merged['Latitude'].combine_first(tabela_resultado['Latitude_escolas_OSM'])
tabela_resultado['Longitude_escolas_OSM']= merged['Longitude'].combine_first(tabela_resultado['Longitude_escolas_OSM'])

display(HTML(tabela_resultado.to_html()))

,Nome do edificio,Flag,Nome_imoveis,Nº_inventario_imoveis,Nome_edificios,Nº_inventario_edificios,Latitude_edificios,Longitude_edificios,Nome_contadores_2,Latitude_contadores_2,Longitude_contadores_2,Nome_baze,Latitude_baze,Longitude_baze,Nome_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM
0,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),I,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),5814,Mercado do Castêlo,5814,,,,,,,,,,,
1,EDIFICIO DO FORUM DA MAIA,I,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,6972,41.234,-8.622233,,,,,,,,,
2,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,I,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,6973,,,,,,,,,,,
3,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,I,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,6975,Paços do Concelho da Maia,6975,41.233341,-8.622518,,,,,,,,,
4,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),I,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),24755,Torre Lidador - Câmara Municipal da Maia,24755,41.233396,-8.622039,,,,,,,,,
5,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),I,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),38754,Complexo Municipal de Piscinas de Gueifães,38754,41.214994,-8.600761,EB de Gueifães,41.214185,-8.600243,,,,,,
6,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",I,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",45234,Creche/ Pré-Escolar de Águas Santas 2,45234,,,,,,,,,,,
7,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",I,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",46678,Polidesportivo Municipal do Lidador,46678,,,,,,,,,,,
8,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),I,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),46679,Polidesportivo Monte Faro,46679,,,,,,,,,,,
9,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",I,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",46680,Polidesportivo Municipal dos Moutidos,46680,,,,,,,,,,,


#### Tabela final

Remoção dos valores que não tiveram matching de nenhuma tabela e tabela final

In [ ]:
tabela_resultado = tabela_resultado[~(tabela_resultado['Nome_imoveis'].eq('') & tabela_resultado['Nome_edificios'].eq('') & tabela_resultado['Nome_contadores_2'].eq('') & tabela_resultado['Nome_baze'].eq(''))]
tabela_resultado = tabela_resultado[~(tabela_resultado['Nome_imoveis'].eq('') & tabela_resultado['Nome_edificios'].eq('') & tabela_resultado['Nome_contadores_2'].eq('') & tabela_resultado['Nome_escolas_OSM'].eq(''))]
tabela_resultado = tabela_resultado[~(tabela_resultado['Nome_imoveis'].eq('') & tabela_resultado['Nome_edificios'].eq('') & tabela_resultado['Nome_baze'].eq('') & tabela_resultado['Nome_escolas_OSM'].eq(''))]
tabela_resultado = tabela_resultado[~(tabela_resultado['Nome_imoveis'].eq('') & tabela_resultado['Nome_contadores_2'].eq('') & tabela_resultado['Nome_baze'].eq('') & tabela_resultado['Nome_escolas_OSM'].eq(''))]
tabela_resultado = tabela_resultado[~(tabela_resultado['Nome_edificios'].eq('') & tabela_resultado['Nome_contadores_2'].eq('') & tabela_resultado['Nome_baze'].eq('') & tabela_resultado['Nome_escolas_OSM'].eq(''))]

display(HTML(tabela_resultado.to_html()))

,Nome do edificio,Flag,Nome_imoveis,Nº_inventario_imoveis,Nome_edificios,Nº_inventario_edificios,Latitude_edificios,Longitude_edificios,Nome_contadores_2,Latitude_contadores_2,Longitude_contadores_2,Nome_baze,Latitude_baze,Longitude_baze,Nome_escolas_OSM,Latitude_escolas_OSM,Longitude_escolas_OSM
0,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),I,MERCADO MUNICIPAL CORONEL MOREIRA (MERCADO CASTELO DA MAIA),5814,Mercado do Castêlo,5814,,,,,,,,,,,
1,EDIFICIO DO FORUM DA MAIA,I,EDIFICIO DO FORUM DA MAIA,6972,Fórum da Maia,6972,41.234,-8.622233,,,,,,,,,
2,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,I,EDIFICIO DAS OFICINAS DE CAMARA MUNICIPAL DA MAIA,6973,Oficinas Gerais da Maia,6973,,,,,,,,,,,
3,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,I,EDIFICIO DA CAMARA MUNICIPAL DA MAIA - PAÇOS DO CONCELHO,6975,Paços do Concelho da Maia,6975,41.233341,-8.622518,,,,,,,,,
4,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),I,TORRE LIDADOR (AMPLIAÇAO DO EDIFICIO ANEXO AOS PAÇOS DO CONCELHO),24755,Torre Lidador - Câmara Municipal da Maia,24755,41.233396,-8.622039,,,,,,,,,
5,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),I,PISCINA MUNICIPAL DE GUEIFAES (COMPLEXO MUNICIPAL DE PISCINAS DE GUEIFAES),38754,Complexo Municipal de Piscinas de Gueifães,38754,41.214994,-8.600761,EB de Gueifães,41.214185,-8.600243,,,,,,
6,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",I,"CRECHE INFANTARIO AGUAS SANTAS II (FRACÇAO AUTONOMA DESIGNADA POR ""FRACÇAO A"", INTEGRADA NO EDIFICIO SITUADO NA RUA JOSE REGIO Nº 242 A 264, DESTINADA A FINS DE CARACTER SOCIAL)-CONTRATO COMODATO SANTA CASA MISERICORDIA DA MAIA,30/07/2018",45234,Creche/ Pré-Escolar de Águas Santas 2,45234,,,,,,,,,,,
7,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",I,"POLIDESPORTIVO MUNICIPAL DO LIDADOR - VILA NOVA DA TELHA, COMPOSTO POR 1 PAR DE BALIZAS 4 TABELAS BASKET",46678,Polidesportivo Municipal do Lidador,46678,,,,,,,,,,,
8,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),I,POLIDESPORTIVO MUNICIPAL DE MONTE FARO (1 PAR DE BALIZAS),46679,Polidesportivo Monte Faro,46679,,,,,,,,,,,
9,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",I,"POLIDESPORTIVO MUNICIPAL DE MOUTIDOS (INSERIDO EB1 DOS MOUTIDOS, 2 BALIZAS E 2 TABELAS)",46680,Polidesportivo Municipal dos Moutidos,46680,,,,,,,,,,,


In [ ]:
filename = 'Tabela_resultado.xlsx'
tabela_resultado.to_excel(filename, index=False)

button = widgets.Button(
    description='Download Tabela_resultado',
    button_style='primary'
    )

def on_button_click(b):
    files.download(filename)

button.on_click(on_button_click)
print("A tabela resultado final: ")
display(button)

A tabela resultado final: 


Button(button_style='primary', description='Download Tabela_resultado', style=ButtonStyle())